# Cascata de LLMs

A tentativa por agentes ainda não apresentou respostas satisfatórias.

Para chegarmos a um resultado e até obter insights para uma abordagens com agentes, preferiu-se uma abordagem que quebra o problema de encontrar os contextos para as perguntas em problemas menores, cada um deles resolvidos por uma LLM cuja a entrada é a tabela resultante do processamento de outra (LLM em cadeia)

Subproblemas:

1. Filtrar perguntas com possibilidade de gabarito errado dado os comentários
1. Separar os elementos da pergunta em Contexto, Assertivas e Alternativas
1. Extrair os artigos citados nos comentários
1. Relacionar os artigos citados com as Assertivas
1. RAG + LLM para completar os contextos da pergunta ou de cada afirmação

# ENV

In [1]:
!pip install langchain-groq langchain faiss-cpu langchain_community sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using ca

# Imports e Defines

In [15]:
import json
import pickle
import pandas as pd
import numpy as np
from getpass import getpass
from tqdm import tqdm
from typing import List
import natsort
import faiss
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [3]:
GROQ_API_KEY = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [139]:
def loop_fn(df, fn, kwarg=None):
    list_of_outputs = []
    for index, row in tqdm(df.iterrows(), total=len(df)):
        try:
            if kwarg:
                output = fn(row, kwarg)
            else:
                output = fn(row)
            print(output)
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            output = "Error"
        list_of_outputs.append(output)
    return list_of_outputs

def edit_docs(docs):
    return f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)])

def edit_docs_clean(docs):
    return "\n".join([d.page_content for d in docs])

def pretty_print_docs(docs):
    print(edit_docs(docs))

def print_only_artID(texto):
    x = ['Art. ' + y.strip().split('. ')[0] for y in texto.replace('\n', '').split('Art. ')][1:]
    natsort.natsorted(x)
    print(x)

def convert_strlist_to_liststr(strlist):
    strlist = strlist[1:-1] # remove []
    strlist = strlist.replace("'", "") # remove '
    liststr = strlist.split(", ") # split elements
    return liststr

# Load LLM

In [5]:
llama3_70b_temp0 = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key=GROQ_API_KEY)

# Test
llama3_70b_temp0.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 17, 'total_tokens': 25, 'completion_time': 0.022857143, 'prompt_time': 0.005614869, 'queue_time': None, 'total_time': 0.028472011999999998}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-9dba912a-5cbd-437e-9b9a-d0e24453637d-0')

# Load Data

In [ ]:
df = pd.read_csv("/content/dataset_concursos.csv")
df.head()

,pergunta,resposta,comentarios,numero_na_captura,id,area,ano,banca,prova
0,"Diego, em razão do falecimento do seu pai Eucl...",C,CC\nArt. 1.826. O possuidor da herança está ob...,1,Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...
1,A sucessão testamentária é um componente impor...,B,I -\nArt. 1.867. Ao cego só se permite o testa...,2,Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,2024,Instituto Consulplan,Instituto Consulplan - DPE PR - Analista - Ár...
2,"Matheus, filho de um milionário, aos dezesseis...",E,A resposta da questão está no Art. 1.880. O te...,3,Q3251846,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RO - Promotor de Justiça Substit...
3,"Mariana, solteira, não convivente em união est...",C,"Art. 1.860. Além dos incapazes, não podem test...",4,Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação Getúlio Vargas - FGV,FGV - MPE GO - Promotor de Justiça Substituto...
4,Lofredo faleceu deixando quatro filhas vivas: ...,E,Essa questão não tem gabarito certo. Explico.\...,5,Q3116405,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...


# Filtragem de perguntas

In [ ]:
class HAS_ANSWER_JSON(BaseModel):
    score: str = Field(description="Score para probabilidade da pergunta não possuir gabarito correto")
    minha_resposta: str = Field(description="Resposta que considero correto")
    gabarito: str = Field(description="Gabarito oficial")
    justificativa_erro: str = Field(description="Por que o gabarito está errado (se estiver) ?")
    justificativa_correto: str = Field(description="Por que sua resposta está correta ?")

output_parser = JsonOutputParser(pydantic_object=HAS_ANSWER_JSON)

In [ ]:
prompt = """
Você é um estudante dedicado de direito do Código Civil Brasileiro. Em especial dos livros IV e V: Direito da Família e Sucessões.
Durante seus estudos você se depara com um conjunto de perguntas de concurso e percebe que algumas delas não possuem gabarito correto.
Para tirar sua dúvida, você olha os comentários de outros estudantes e resolve quais perguntas não possuem gabarito correto.
Como nem sempre é claro você constroi a seguinte métrica:

Score 0 - O gabarito não está correto.
Score 1 - Talvez o gabarito esteja correto.
Score 2 - O gabarito está correto.

Continue fazendo isso para todas as perguntas.

Pergunta: {pergunta}
Comentários: {comentarios}
Gabarito: {gabarito}

Instruções de saída: {format_instructions}
"""

prompt_template = PromptTemplate(
    input_variables=["pergunta", "comentarios", "gabarito"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [ ]:
def call_has_answer(sample):
    output = chain.invoke({
        "pergunta": sample["pergunta"],
        "comentarios": sample["comentarios"],
        "gabarito": sample["resposta"],
        })
    return output

# test
call_has_answer(df.iloc[0])

{'score': '0',
 'minha_resposta': 'B',
 'gabarito': 'C',
 'justificativa_erro': 'O gabarito oficial está errado porque não considera que o possuidor de boa-fé tem direito a todos os frutos já percebidos e aos que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos, além de ter direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção.',
 'justificativa_correto': 'A resposta correta é B porque o art. 1.214 do CC estabelece que o possuidor de boa-fé tem direito aos frutos percebidos, e o art. 1.219 estabelece que o possuidor de boa-fé tem direito à indenização das benfeitorias necessárias e úteis, bem como, quanto às voluptuárias, se não lhe forem pagas, a levantá-las, quando o puder sem detrimento da coisa, e poderá exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.'}

## Loop

In [ ]:
list_of_outputs = []
for index, row in tqdm(df.iterrows(), total=len(df)):
    try:
        output = call_has_answer(row)
        print(output)
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        output = "Error"
    list_of_outputs.append(output)

  2%|▎         | 1/40 [00:01<00:56,  1.44s/it]

{'score': '0', 'minha_resposta': 'B', 'gabarito': 'C', 'justificativa_erro': 'O gabarito oficial está errado porque não considera que o possuidor de boa-fé tem direito a todos os frutos já percebidos e aos que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos, além de ter direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção.', 'justificativa_correto': 'A resposta correta é B porque o art. 1.214 do CC estabelece que o possuidor de boa-fé tem direito aos frutos percebidos, e o art. 1.219 estabelece que o possuidor de boa-fé tem direito à indenização das benfeitorias necessárias e úteis, bem como, quanto às voluptuárias, se não lhe forem pagas, a levantá-las, quando o puder sem detrimento da coisa, e poderá exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.'}


  5%|▌         | 2/40 [00:15<05:31,  8.73s/it]

{'score': '2', 'minha_resposta': 'B', 'gabarito': 'B', 'justificativa_erro': '', 'justificativa_correto': 'A afirmativa I está correta de acordo com o art. 1.867 do Código Civil Brasileiro. A afirmativa IV também está correta de acordo com o art. 1.880 do Código Civil Brasileiro.'}


  8%|▊         | 3/40 [00:30<07:05, 11.49s/it]

...

{'score': '2', 'minha_resposta': 'C', 'gabarito': 'C', 'justificativa_erro': '', 'justificativa_correto': 'O art. 1.857 do CC estabelece que a legítima dos herdeiros necessários não poderá ser incluída no testamento, o que confirma a resposta certa.'}


 98%|█████████▊| 39/40 [08:35<00:15, 15.66s/it]

{'score': '0', 'minha_resposta': 'A', 'gabarito': 'E', 'justificativa_erro': 'O gabarito está errado porque a contagem do prazo prescricional para a petição de herança começa a partir da abertura da sucessão, e não da prolação de julgamento definitivo em investigação de paternidade.', 'justificativa_correto': 'A ausência de prévia propositura de ação de investigação de paternidade não constitui óbice para o ajuizamento de ação de petição de herança e para o início da contagem do prazo prescricional.'}


100%|██████████| 40/40 [08:45<00:00, 13.14s/it]

{'score': '2', 'minha_resposta': 'C', 'gabarito': 'C', 'justificativa_erro': '', 'justificativa_correto': 'O art. 1.798 do Código Civil estabelece que os nascituros têm legitimidade para suceder, desde que tenham sido concebidos no momento da abertura da sucessão. No caso em questão, Ana estava grávida quando da morte de João, portanto o nascituro tem legitimidade para suceder.'}


In [ ]:
for idx, output in enumerate(list_of_outputs):
    if output == "Error":
        new_out = call_has_answer(df.iloc[idx])

In [ ]:
check_gabarito_df = pd.DataFrame(list_of_outputs)
check_gabarito_df.head()

,score,minha_resposta,gabarito,justificativa_erro,justificativa_correto
0,0,B,C,O gabarito oficial está errado porque não cons...,A resposta correta é B porque o art. 1.214 do ...
1,2,B,B,,A afirmativa I está correta de acordo com o ar...
2,2,E,E,,"O gabarito está correto, pois o Art. 1.880 do ..."
3,2,D,C,O gabarito oficial está errado porque a incapa...,A resposta correta é D porque a incapacidade r...
4,0,Não há resposta correta,E,O gabarito está errado porque a esposa Afrodit...,A resposta correta é que as filhas receberiam ...


In [ ]:
check_gabarito_df.to_csv("check_gabarito_df.csv", index=False)

In [ ]:
check_gabarito_df["resposta_diferentes"] = check_gabarito_df["minha_resposta"] != check_gabarito_df["gabarito"]
check_gabarito_df[check_gabarito_df["resposta_diferentes"] == True]

,score,minha_resposta,gabarito,justificativa_erro,justificativa_correto,resposta_diferentes
0,0,B,C,O gabarito oficial está errado porque não cons...,A resposta correta é B porque o art. 1.214 do ...,True
3,2,D,C,O gabarito oficial está errado porque a incapa...,A resposta correta é D porque a incapacidade r...,True
4,0,Não há resposta correta,E,O gabarito está errado porque a esposa Afrodit...,A resposta correta é que as filhas receberiam ...,True
5,0,A,C,O gabarito oficial está errado porque a preexi...,A configuração da união estável entre Rogério ...,True
6,2,C,E,O gabarito oficial está errado porque o item e...,"O item está correto porque, de acordo com o ar...",True
8,2,C,E,"O gabarito oficial está errado, pois o art. 27...","A resposta certa é C, pois o art. 270 do Códig...",True
10,2,E,D,O gabarito oficial está errado porque o art. 1...,A resposta correta é E) deverá se habilitar no...,True
16,2,C,E,A questão está errada pois não considera a pos...,"De acordo com o art. 1.798 do Código Civil, as...",True
22,0,D,B,O gabarito está errado porque a assertiva III ...,"A resposta correta é D, pois todas as assertiv...",True
24,0,C,E,O gabarito oficial está errado porque a exclus...,A resposta correta é a letra C porque Pedro po...,True


In [ ]:
sem_gabarito_ids = [4]
df = df.drop(sem_gabarito_ids)
df.reset_index(drop=True, inplace=True)
df.head()

,pergunta,resposta,comentarios,numero_na_captura,id,area,ano,banca,prova
0,"Diego, em razão do falecimento do seu pai Eucl...",C,CC\nArt. 1.826. O possuidor da herança está ob...,1,Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...
1,A sucessão testamentária é um componente impor...,B,I -\nArt. 1.867. Ao cego só se permite o testa...,2,Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,2024,Instituto Consulplan,Instituto Consulplan - DPE PR - Analista - Ár...
2,"Matheus, filho de um milionário, aos dezesseis...",E,A resposta da questão está no Art. 1.880. O te...,3,Q3251846,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RO - Promotor de Justiça Substit...
3,"Mariana, solteira, não convivente em união est...",C,"Art. 1.860. Além dos incapazes, não podem test...",4,Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação Getúlio Vargas - FGV,FGV - MPE GO - Promotor de Justiça Substituto...
4,Rogério é casado com Cláudio e ambos são pais ...,C,Gabarito C.\n\nA preexistência de casamento ou...,6,Q3192677,Direito Civil Direito de Família (Art. 1.511 ...,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RJ - Promotor de Justiça Substit...


In [ ]:
len(df)

39

In [ ]:
df['duvidou_do_gabarito'] = check_gabarito_df.drop(sem_gabarito_ids).reset_index()["resposta_diferentes"]
df[df["duvidou_do_gabarito"] == True]

,pergunta,resposta,comentarios,numero_na_captura,id,area,ano,banca,prova,duvidou_do_gabarito
0,"Diego, em razão do falecimento do seu pai Eucl...",C,CC\nArt. 1.826. O possuidor da herança está ob...,1,Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...,True
3,"Mariana, solteira, não convivente em união est...",C,"Art. 1.860. Além dos incapazes, não podem test...",4,Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação Getúlio Vargas - FGV,FGV - MPE GO - Promotor de Justiça Substituto...,True
4,Rogério é casado com Cláudio e ambos são pais ...,C,Gabarito C.\n\nA preexistência de casamento ou...,6,Q3192677,Direito Civil Direito de Família (Art. 1.511 ...,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RJ - Promotor de Justiça Substit...,True
5,"João e Maria, amigos de longa data, decidem em...",E,"GABARITO CERTO, acredito que esteja errado.\nA...",7,Q3231292,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Instituto Consulplan,Instituto Consulplan - MPE SC - Promotor de J...,True
7,"João e Maria, amigos de longa data, decidem em...",E,"Código Civil, art. 270: Se um dos credores sol...",9,Q3231293,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Instituto Consulplan,Instituto Consulplan - MPE SC - Promotor de J...,True
9,"O Banco XYZ é credor de Diana, que não possui ...",D,CC:\nArt. 1.813: Quando o renunciante prejudic...,11,Q3071802,Direito Civil Direito das Sucessões (Art. 1.7...,2023,Fundação Getúlio Vargas - FGV,FGV - Câmara dos Deputados - Analista Legisla...,True
15,"João, nascido em Brasília – DF, viveu toda a s...",E,Art. 1.798. Legitimam-se a suceder as pessoas ...,17,Q2802283,Direito Civil Direito das Sucessões (Art. 1.7...,2023,Centro de Seleção e de Promoção de Eventos Un...,CESPE/CEBRASPE - MPE SC - Promotor de Justiça...,True
21,A cessão de direitos hereditários é ato permit...,B,"Art. 1.793. O direito à sucessão aberta, bem c...",23,Q3034664,Direito Civil Direito das Sucessões (Art. 1.7...,2023,Instituto de Estudos Superiores do Extremo Su...,IESES - TJ AM - Titular de Serviços Notariais...,True
23,"Ana faleceu deixando dois filhos, Pedro e Paul...",E,a) Herdeiros necessários podem ser excluídos d...,25,Q2800095,Direito Civil Exclusão da Sucessão,2023,"Centro de Extensão, Treinamento e Aperfeiçoam...",CETAP - IGEPREV - Técnico Previdenciário - 2023,True
29,Mariana falece em outubro de 2020 sem deixar t...,B,Marcos: não há nenhum impedimento p/ a sucessã...,31,Q3028625,Direito Civil Direito das Sucessões (Art. 1.7...,2023,Fundação Getúlio Vargas - FGV,FGV - DPE RJ - Defensor Público de Classe Ini...,True


In [ ]:
df.to_csv("dataset_concurso_filtered.csv", index=False)

# Quebra da questão e Classificação com base em Alternativas

- Questões: (Enunciado, Assetivas e Alternativas)
- Alternativas: (alternativas com conteúdo x alternativa que completam o enunciado)

In [ ]:
class QUESTION_SPLIT_JSON(BaseModel):
    enunciado: str = Field(description="Enunciado da questão")
    assertivas: List[str] = Field(description="Assertivas da questão")
    alternativas: List[str] = Field(description="Alternativas da questão")
    classificacao: int = Field(description="Classificação da questão")
    notas: str = Field(description="Raciocínio e observações usadas")

output_parser = JsonOutputParser(pydantic_object=QUESTION_SPLIT_JSON)

In [ ]:
prompt = """
Você é um estudante dedicado de direito do Código Civil Brasileiro. Em especial dos livros IV e V: Direito da Família e Sucessões.
Após encontrar um conjunto de questões Você deseja montar uma tabela separando as parte das perguntas em:

- Enunciado: Texto inicial das perguntas
- Assertivas: Assertivas a serem julgadas como Verdadeiro ou Falso, iniciadas com algoritmos romanos (I, II, ...) ou por parenteses "()".
- Alternativas: Alternativas que completam o enunciado e da qual a resposta é obtida, são identificadas por letras maiúsculas ("A)", "B)", "C)", ...)

Nem toda questão possui assertivas, mas todas possuem alternativas. Caso não possua assertiva preencha com ''.

Além disso, você percebe que é útil classificar se as alternativas possuem conteúdo para ser avaliado ou não.
Alternativas que não possuem conteúdos para serem avaliados são mais curtas e vão ter um dos seguintes formatos:
"A) I e II" ou "E) Todos os itens estão certos.".
Alguma alternativas seu contéudo não é auto contido, nesse caso são complementares ao questionamento feito no enunciado.
Exemplos desse tipo são: 'C) Marcos e Augusto;', "C) Certo" ou "D) Adultério."
Atribua um único valor de 0 para quando as alternativas que não possuem conteúdo;
o valor de 1 para quando as alternativas que possuem conteúdo autocontido;
e 2 para quando as alternativas que possuem conteúdo curto e altamente dependente do enunciado.

Pergunta: {pergunta}

Instruções de saída: {format_instructions}
""".strip()

prompt_template = PromptTemplate(
    input_variables=["pergunta"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [ ]:
def call_split_question(sample):
    output = chain.invoke({
        "pergunta": sample["pergunta"],
        })
    return output

# test
call_split_question(df.iloc[3])

{'enunciado': 'Mariana, solteira, não convivente em união estável, relativamente incapaz em razão da idade, sem filhos e sem ascendentes vivos, elaborou, de próprio punho, testamento particular, o qual foi lido por ela na presença de três testemunhas que o subscreveram, dispondo sobre a destinação post mortem de todos os bens integrantes de seu patrimônio.',
 'assertivas': [],
 'alternativas': ['A) O testamento de Mariana, em razão da sua incapacidade civil no momento da sua elaboração, independentemente da sua capacidade no momento da abertura da sucessão, é anulável, assim como a deixa em favor de Túlio é nula, por ser ele presumidamente pessoa interposta a não legitimado a suceder.',
  'B) A incapacidade relativa de Mariana no momento da elaboração do testamento afeta a validade do ato, mas a deixa em favor de Clara constitui legado que, em razão da abusividade dos gravames impostos, será tido como puro e simples.',
  'C) A incapacidade relativa de Mariana no momento da elaboração d

## Loop

In [ ]:
list_of_outputs = []
for index, row in tqdm(df.iterrows(), total=len(df)):
    try:
        output = call_split_question(row)
        print(output)
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        output = "Error"
    list_of_outputs.append(output)

  3%|▎         | 1/39 [00:02<01:38,  2.60s/it]

{'enunciado': 'Diego, em razão do falecimento do seu pai Euclides, viúvo, herdou um pequeno sítio localizado em Botucatu, único bem deixado por Euclides. Após a realização de todos os trâmites legais e transferida a propriedade, Diego realizou uma série de benfeitorias no sítio, que estava em situação muito precária, e passou a locá-lo para eventos corporativos, o que se tornou a principal fonte de renda de Diego. Passados 5 (cinco) anos, Diego é surpreendido com uma citação em ação nulidade de inventário cumulada com petição de herança, promovida por João Carlos, que logrou êxito em provar que também era filho de Euclides, anexando a sentença transitada em julgado da ação de investigação de paternidade, fato que nunca foi e nem poderia ser de conhecimento de Diego.', 'assertivas': [], 'alternativas': ['A) todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização

  5%|▌         | 2/39 [00:04<01:15,  2.05s/it]

{'enunciado': 'A sucessão testamentária é um componente importante do planejamento patrimonial e permite que as pessoas tenham controle sobre o destino de seus bens após a morte. É fundamental que um testamento seja elaborado de forma cuidadosa e de acordo com as leis locais para garantir que a vontade do testador seja respeitada e que a sucessão ocorra de maneira adequada. Em relação à sucessão testamentária, analise as afirmativas a seguir.', 'assertivas': ['I. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.', 'II. O testamento cerrado deve ser escrito mecanicamente, desde que seu subscritor numere e autentique, com a sua assinatura, todas as páginas.', 'III. Em circunstâncias excepcionais declaradas na cédula, o testamento particular de próprio punho e assinado pelo testador, sem te

  8%|▊         | 3/39 [00:06<01:09,  1.93s/it]

...

{'enunciado': 'A legítima dos herdeiros necessários não poderá ser incluída no testamento.', 'assertivas': [], 'alternativas': ['C) Certo', 'E) Errado'], 'classificacao': 1, 'notas': ''}


 97%|█████████▋| 38/39 [07:47<00:14, 14.25s/it]

{'enunciado': 'Renato, nascido em 20/2/1985, ajuizou ação de investigação de paternidade post mortem cumulada com petição de herança em desfavor de Mariana e de Juliana, filhas herdeiras de Manoel, indicado como suposto pai biológico do autor. Na petição inicial, protocolada em 15/12/2022, Renato demonstrou que, no dia 5/2/2010, ocorreu o falecimento de Manoel e informou que, apenas posteriormente, optou por tomar medida jurídica para que houvesse o reconhecimento da paternidade e a restituição de herança. Nessa situação hipotética, a pretensão de Renato referente à petição de herança', 'assertivas': [], 'alternativas': ['A) não está prescrita, porque ainda não se consumou o prazo prescricional, que deve ser contado da abertura da sucessão.', 'B) pode ser exercida, por ser imprescritível.', 'C) somente pode ser apresentada após a prolação de julgamento definitivo em investigação de paternidade, em cuja data se inicia a contagem do prazo prescricional para petição de herança.', 'D) some

100%|██████████| 39/39 [08:01<00:00, 12.34s/it]

{'enunciado': 'João, nascido em Brasília – DF, viveu toda a sua vida em Penha – SC e morreu em Florianópolis – SC. Quando ele morreu, Ana, sua esposa, estava grávida. Considerando essa situação hipotética e o disposto no Código Civil sobre o direito das sucessões, julgue os itens subsequentes.', 'assertivas': [], 'alternativas': ['C) Certo', 'E) Errado'], 'classificacao': 1, 'notas': 'A assertiva está correta, pois o nascituro tem legitimidade para suceder, de acordo com o art. 1.798 do Código Civil.'}


In [ ]:
question_split_df = pd.DataFrame(list_of_outputs)
question_split_df.head()

,enunciado,assertivas,alternativas,classificacao,notas
0,"Diego, em razão do falecimento do seu pai Eucl...",[],"[A) todos os frutos já percebidos, bem como aq...",1,A questão trata da proteção do herdeiro aparen...
1,A sucessão testamentária é um componente impor...,[I. Ao cego só se permite o testamento público...,"[A) I e II., B) I e IV., C) II e IV., D) III e...",0,A classificação 0 foi atribuída pois as altern...
2,"Matheus, filho de um milionário, aos dezesseis...",[],[A) O codicilo só é válido se não houver testa...,1,A questão trata da validade do testamento e da...
3,"Mariana, solteira, não convivente em união est...",[],"[A) O testamento de Mariana, em razão da sua i...",1,A questão aborda a validade do testamento de M...
4,Rogério é casado com Cláudio e ambos são pais ...,[],[A) é possível a configuração da união estável...,1,A questão envolve a configuração da união está...


In [ ]:
question_split_df.to_csv("question_split_df.csv", index=False)

Após análise, a classificação precisou ser corrigida. E salva no arquivo question_split_df_manual.csv

In [ ]:
question_split_df = pd.read_csv("question_split_df_manual.csv")
question_split_df.head()

,enunciado,assertivas,alternativas,classificacao,notas
0,"Diego, em razão do falecimento do seu pai Eucl...",[],"['A) todos os frutos já percebidos, bem como a...",1,A questão trata da proteção do herdeiro aparen...
1,A sucessão testamentária é um componente impor...,['I. Ao cego só se permite o testamento públic...,"['A) I e II.', 'B) I e IV.', 'C) II e IV.', 'D...",0,A classificação 0 foi atribuída pois as altern...
2,"Matheus, filho de um milionário, aos dezesseis...",[],['A) O codicilo só é válido se não houver test...,1,A questão trata da validade do testamento e da...
3,"Mariana, solteira, não convivente em união est...",[],"['A) O testamento de Mariana, em razão da sua ...",1,A questão aborda a validade do testamento de M...
4,Rogério é casado com Cláudio e ambos são pais ...,[],['A) é possível a configuração da união estáve...,1,A questão envolve a configuração da união está...


In [ ]:
df = pd.concat([df, question_split_df.drop(['notas'], axis=1)], axis=1)
df.to_csv("dataset_concurso_qsplit.csv", index=False)
df.head()

,pergunta,resposta,comentarios,numero_na_captura,id,area,ano,banca,prova,duvidou_do_gabarito,enunciado,assertivas,alternativas,classificacao
0,"Diego, em razão do falecimento do seu pai Eucl...",C,CC\nArt. 1.826. O possuidor da herança está ob...,1,Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...,True,"Diego, em razão do falecimento do seu pai Eucl...",[],"['A) todos os frutos já percebidos, bem como a...",1
1,A sucessão testamentária é um componente impor...,B,I -\nArt. 1.867. Ao cego só se permite o testa...,2,Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,2024,Instituto Consulplan,Instituto Consulplan - DPE PR - Analista - Ár...,False,A sucessão testamentária é um componente impor...,['I. Ao cego só se permite o testamento públic...,"['A) I e II.', 'B) I e IV.', 'C) II e IV.', 'D...",0
2,"Matheus, filho de um milionário, aos dezesseis...",E,A resposta da questão está no Art. 1.880. O te...,3,Q3251846,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RO - Promotor de Justiça Substit...,False,"Matheus, filho de um milionário, aos dezesseis...",[],['A) O codicilo só é válido se não houver test...,1
3,"Mariana, solteira, não convivente em união est...",C,"Art. 1.860. Além dos incapazes, não podem test...",4,Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação Getúlio Vargas - FGV,FGV - MPE GO - Promotor de Justiça Substituto...,True,"Mariana, solteira, não convivente em união est...",[],"['A) O testamento de Mariana, em razão da sua ...",1
4,Rogério é casado com Cláudio e ambos são pais ...,C,Gabarito C.\n\nA preexistência de casamento ou...,6,Q3192677,Direito Civil Direito de Família (Art. 1.511 ...,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RJ - Promotor de Justiça Substit...,True,Rogério é casado com Cláudio e ambos são pais ...,[],['A) é possível a configuração da união estáve...,1


# Extração de artigos dos comentários

In [ ]:
class COMMENTS_ARTICLES_JSON(BaseModel):
    artigos: List[str] = Field(description="Lista dos ids dos artigos citados nos comentários")

output_parser = JsonOutputParser(pydantic_object=COMMENTS_ARTICLES_JSON)

In [ ]:
prompt = """
Você é um estudante dedicado de direito do Código Civil Brasileiro.
Após responder algumas questões de concurso, você decide olhar os comentários de outros alunos sobre as questões.
Nos comentários a diversas referências a artigos do Código Civil.
Extraia os identificadores dos artigos citados nos comentários para lê-los posteriormente.
Use a formatação: Art. id do Código Civil.

Não se esqueça que os artigos podem possuir parágrafos (PARÁGRAFO ÚNICO,  §2°, ...) e incisos (I, II, ...).
Como pode haver artigos de outros códigos nesses casos deixe indicado a que código ou resolução pertence.

Comentários: {comentarios}

Instruções de saída: {format_instructions}
""".strip()

prompt_template = PromptTemplate(
    input_variables=["comentarios"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [ ]:
def call_comment_articles_question(sample):
    output = chain.invoke({
        "comentarios": sample["comentarios"],
        })
    return output

# test
call_comment_articles_question(df.iloc[1])

{'artigos': ['Art. 1.867 do Código Civil',
  'Art. 1.868 do Código Civil',
  'Art. 1.868, Parágrafo único do Código Civil',
  'Art. 1.879 do Código Civil',
  'Art. 1.880 do Código Civil']}

## Loop

In [ ]:
list_of_outputs = loop_fn(df, call_comment_articles_question)

  3%|▎         | 1/39 [00:00<00:33,  1.13it/s]

{'artigos': ['Art. 1.826', 'Art. 1.214', 'Art. 1.214, Parágrafo único', 'Art. 1.219', 'Art. 1.973', 'Art. 1.974', 'Art. 1.975', 'Art. 205', 'Art. 206']}


  5%|▌         | 2/39 [00:01<00:27,  1.36it/s]

{'artigos': ['Art. 1.867 do Código Civil', 'Art. 1.868 do Código Civil', 'Art. 1.868, Parágrafo único do Código Civil', 'Art. 1.879 do Código Civil', 'Art. 1.880 do Código Civil']}


  8%|▊         | 3/39 [00:02<00:30,  1.18it/s]

...

{'artigos': ['Art. 1.857 do CC']}


 97%|█████████▋| 38/39 [04:20<00:09,  9.99s/it]

{'artigos': ['Art. 205 do CC', 'Art. 1.822 do CC', 'Súmula 149 do STF']}


100%|██████████| 39/39 [04:26<00:00,  6.82s/it]

{'artigos': ['Art. 1.798 do Código Civil', 'Art. 1.799 do Código Civil']}


In [ ]:
df_comments_articles = pd.DataFrame(list_of_outputs)
df_comments_articles['comments'] = df['comentarios']
df_comments_articles.head()

,artigos,comments
0,"[Art. 1.826, Art. 1.214, Art. 1.214, Parágrafo...",CC\nArt. 1.826. O possuidor da herança está ob...
1,"[Art. 1.867 do Código Civil, Art. 1.868 do Cód...",I -\nArt. 1.867. Ao cego só se permite o testa...
2,"[Art. 1.880, Art. 1.860, Art. 1.860 Parágrafo ...",A resposta da questão está no Art. 1.880. O te...
3,"[Art. 1.860, Art. 1.799, Art. 1.900, Art. 1.80...","Art. 1.860. Além dos incapazes, não podem test..."
4,"[Art. 1.723, Art. 1.723, § 1º, Art. 1.521]",Gabarito C.\n\nA preexistência de casamento ou...


In [ ]:
df_comments_articles.to_csv("df_comments_articles.csv", index=False)

In [ ]:
df = pd.read_csv("dataset_concurso_qsplit.csv")
df_comments_articles = pd.read_csv("df_comments_articles.csv")
df['artigos_nos_comentarios'] = df_comments_articles['artigos']
df.head()

,pergunta,resposta,comentarios,numero_na_captura,id,area,ano,banca,prova,duvidou_do_gabarito,enunciado,assertivas,alternativas,classificacao,artigos_nos_comentarios
0,"Diego, em razão do falecimento do seu pai Eucl...",C,CC\nArt. 1.826. O possuidor da herança está ob...,1,Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...,True,"Diego, em razão do falecimento do seu pai Eucl...",[],"['A) todos os frutos já percebidos, bem como a...",1,"['Art. 1.826', 'Art. 1.214', 'Art. 1.214, Pará..."
1,A sucessão testamentária é um componente impor...,B,I -\nArt. 1.867. Ao cego só se permite o testa...,2,Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,2024,Instituto Consulplan,Instituto Consulplan - DPE PR - Analista - Ár...,False,A sucessão testamentária é um componente impor...,['I. Ao cego só se permite o testamento públic...,"['A) I e II.', 'B) I e IV.', 'C) II e IV.', 'D...",0,"['Art. 1.867 do Código Civil', 'Art. 1.868 do ..."
2,"Matheus, filho de um milionário, aos dezesseis...",E,A resposta da questão está no Art. 1.880. O te...,3,Q3251846,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RO - Promotor de Justiça Substit...,False,"Matheus, filho de um milionário, aos dezesseis...",[],['A) O codicilo só é válido se não houver test...,1,"['Art. 1.880', 'Art. 1.860', 'Art. 1.860 Parág..."
3,"Mariana, solteira, não convivente em união est...",C,"Art. 1.860. Além dos incapazes, não podem test...",4,Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação Getúlio Vargas - FGV,FGV - MPE GO - Promotor de Justiça Substituto...,True,"Mariana, solteira, não convivente em união est...",[],"['A) O testamento de Mariana, em razão da sua ...",1,"['Art. 1.860', 'Art. 1.799', 'Art. 1.900', 'Ar..."
4,Rogério é casado com Cláudio e ambos são pais ...,C,Gabarito C.\n\nA preexistência de casamento ou...,6,Q3192677,Direito Civil Direito de Família (Art. 1.511 ...,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RJ - Promotor de Justiça Substit...,True,Rogério é casado com Cláudio e ambos são pais ...,[],['A) é possível a configuração da união estáve...,1,"['Art. 1.723', 'Art. 1.723, § 1º', 'Art. 1.521']"


In [ ]:
df.to_csv("dataset_concurso_coment_artigos.csv", index=False)

# Validação do slip dos artigos citados

In [140]:
class COMMENT_ART_VALIDATION_JSON(BaseModel):
    foi_citado: str = Field(description="O artigo foi citado nos comentarios: Sim or Não")
    trechos: List[str] = Field(description="Trechos do que referenciam o artigo")

output_parser = JsonOutputParser(pydantic_object=COMMENT_ART_VALIDATION_JSON)

In [141]:
prompt = """
Responda se o artigo foi citado nos comentários.
Use somente as palavras Sim e Não.

Além disso, copie os trechos dos comentários que referenciam o artigo.
Copie no máximo 7 palavras para o trecho.

artigo: {artigo}
comentarios: {comentarios}

Instruções de saída: {format_instructions}
""".strip()

prompt_template = PromptTemplate(
    input_variables=["artigo", "comentarios"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [142]:
def call_valid_article_comment(sample):
    sample_output = []
    if sample["artigos_completos"]:
        for artigo in sample["artigos_completos"]:
            output = chain.invoke({
                "artigo": artigo,
                "comentarios": sample["comentarios"],
            })
            sample_output.append(output)
    return sample_output

# test
call_valid_article_comment(df.iloc[0])

[{'foi_citado': 'Sim', 'trechos': ['Art. 1.826.', 'Art. 1.214', 'Art. 1.219']},
 {'foi_citado': 'Sim',
  'trechos': ['observado o disposto nos arts.',
   'O possuidor de boa-fé tem direito',
   'O possuidor de boa-fé tem direito']},
 {'foi_citado': 'Sim',
  'trechos': ['observado o disposto nos arts.',
   'O possuidor de boa-fé tem direito',
   'O possuidor de boa-fé tem direito']},
 {'foi_citado': 'Sim',
  'trechos': ['O possuidor de boa-fé', 'O possuidor de boa-fé']},
 {'foi_citado': 'Sim', 'trechos': ['Art. 1.973.']},
 {'foi_citado': 'Sim',
  'trechos': ['STF já decidiu que o rompimento do testamento só ocorre']},
 {'foi_citado': 'Sim',
  'trechos': ['Art. 1.975. Não se rompe', 'Obs: Nesse caso, o testamento']},
 {'foi_citado': 'Sim',
  'trechos': ['Na hipótese de nulidade de inventário, hipótese da questão, o prazo prescricional é de 10 (dez) anos, salvo melhor juízo. Isso porque, por dedução, nulidade de inventário não encontra previsão nas hipóteses do art. 206. Logo: Art. 205.']

# Relacionar cada alternativa com um conjunto de artigos baseado nos comentários

## Preparando artigos

In [ ]:
df = pd.read_csv("dataset_concurso_coment_artigos.csv")
df.head()

,pergunta,resposta,comentarios,numero_na_captura,id,area,ano,banca,prova,duvidou_do_gabarito,enunciado,assertivas,alternativas,classificacao,artigos_nos_comentarios
0,"Diego, em razão do falecimento do seu pai Eucl...",C,CC\nArt. 1.826. O possuidor da herança está ob...,1,Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...,True,"Diego, em razão do falecimento do seu pai Eucl...",[],"['A) todos os frutos já percebidos, bem como a...",1,"['Art. 1.826', 'Art. 1.214', 'Art. 1.214, Pará..."
1,A sucessão testamentária é um componente impor...,B,I -\nArt. 1.867. Ao cego só se permite o testa...,2,Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,2024,Instituto Consulplan,Instituto Consulplan - DPE PR - Analista - Ár...,False,A sucessão testamentária é um componente impor...,['I. Ao cego só se permite o testamento públic...,"['A) I e II.', 'B) I e IV.', 'C) II e IV.', 'D...",0,"['Art. 1.867 do Código Civil', 'Art. 1.868 do ..."
2,"Matheus, filho de um milionário, aos dezesseis...",E,A resposta da questão está no Art. 1.880. O te...,3,Q3251846,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RO - Promotor de Justiça Substit...,False,"Matheus, filho de um milionário, aos dezesseis...",[],['A) O codicilo só é válido se não houver test...,1,"['Art. 1.880', 'Art. 1.860', 'Art. 1.860 Parág..."
3,"Mariana, solteira, não convivente em união est...",C,"Art. 1.860. Além dos incapazes, não podem test...",4,Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação Getúlio Vargas - FGV,FGV - MPE GO - Promotor de Justiça Substituto...,True,"Mariana, solteira, não convivente em união est...",[],"['A) O testamento de Mariana, em razão da sua ...",1,"['Art. 1.860', 'Art. 1.799', 'Art. 1.900', 'Ar..."
4,Rogério é casado com Cláudio e ambos são pais ...,C,Gabarito C.\n\nA preexistência de casamento ou...,6,Q3192677,Direito Civil Direito de Família (Art. 1.511 ...,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RJ - Promotor de Justiça Substit...,True,Rogério é casado com Cláudio e ambos são pais ...,[],['A) é possível a configuração da união estáve...,1,"['Art. 1.723', 'Art. 1.723, § 1º', 'Art. 1.521']"


In [6]:
with open("/content/codigo_civil.json", "r") as f:
    codigo = json.load(f)

def collect_artigo_and_header(data):
    result = {'articles': [], 'contexts': []}

    def traverse(data):
        if isinstance(data, dict):
            for key, value in data.items():
                if key == 'artigos' and value:
                    result['articles'].append(value)
                if key == 'header' and value:
                    result['contexts'].append(value)
                # Recursive call to handle nested dictionaries
                if isinstance(value, dict):
                    traverse(value)
                elif isinstance(value, list):
                    traverse(value)
        elif isinstance(data, list):
            for item in data:
                # Recursive call to handle lists of dictionaries
                traverse(item)

    traverse(data)
    return result

artigos = collect_artigo_and_header(codigo)['articles']

In [7]:
def get_article(art_id, artigos=artigos):
    for art_group in artigos:
        for artigo in art_group:
            if art_id.replace('.', '') in artigo[:10].replace('.', ''): # limit to get article name in init
                return artigo.replace('\n\n', '').strip()

def get_all_article_citation(art_id, artigos):
    citations = []
    for art_group in artigos:
        for artigo in art_group:
            if art_id in artigo:
                citations.append(artigo)
    return citations

def get_article_group(art_id, artigos):
    for art_group in artigos:
        for artigo in art_group:
            if art_id in artigo:
                return art_group

get_article('Art. 1593')

'Art. 1.593. O parentesco é natural ou civil, conforme resulte de consangüinidade ou outra origem.'

In [ ]:
df['artigos_nos_comentarios'] = df['artigos_nos_comentarios'].apply(convert_strlist_to_liststr)
df.head()

,pergunta,resposta,comentarios,numero_na_captura,id,area,ano,banca,prova,duvidou_do_gabarito,enunciado,assertivas,alternativas,classificacao,artigos_nos_comentarios
0,"Diego, em razão do falecimento do seu pai Eucl...",C,CC\nArt. 1.826. O possuidor da herança está ob...,1,Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...,True,"Diego, em razão do falecimento do seu pai Eucl...",[],"['A) todos os frutos já percebidos, bem como a...",1,"[Art. 1.826, Art. 1.214, Art. 1.214, Parágrafo..."
1,A sucessão testamentária é um componente impor...,B,I -\nArt. 1.867. Ao cego só se permite o testa...,2,Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,2024,Instituto Consulplan,Instituto Consulplan - DPE PR - Analista - Ár...,False,A sucessão testamentária é um componente impor...,['I. Ao cego só se permite o testamento públic...,"['A) I e II.', 'B) I e IV.', 'C) II e IV.', 'D...",0,"[Art. 1.867 do Código Civil, Art. 1.868 do Cód..."
2,"Matheus, filho de um milionário, aos dezesseis...",E,A resposta da questão está no Art. 1.880. O te...,3,Q3251846,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RO - Promotor de Justiça Substit...,False,"Matheus, filho de um milionário, aos dezesseis...",[],['A) O codicilo só é válido se não houver test...,1,"[Art. 1.880, Art. 1.860, Art. 1.860 Parágrafo ..."
3,"Mariana, solteira, não convivente em união est...",C,"Art. 1.860. Além dos incapazes, não podem test...",4,Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação Getúlio Vargas - FGV,FGV - MPE GO - Promotor de Justiça Substituto...,True,"Mariana, solteira, não convivente em união est...",[],"['A) O testamento de Mariana, em razão da sua ...",1,"[Art. 1.860, Art. 1.799, Art. 1.900, Art. 1.80..."
4,Rogério é casado com Cláudio e ambos são pais ...,C,Gabarito C.\n\nA preexistência de casamento ou...,6,Q3192677,Direito Civil Direito de Família (Art. 1.511 ...,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RJ - Promotor de Justiça Substit...,True,Rogério é casado com Cláudio e ambos são pais ...,[],['A) é possível a configuração da união estáve...,1,"[Art. 1.723, Art. 1.723, § 1º, Art. 1.521]"


In [ ]:
df.loc[:, 'artigos_nos_comentarios']

0     [Art. 1.826, Art. 1.214, Art. 1.214, Parágrafo...
1     [Art. 1.867 do Código Civil, Art. 1.868 do Cód...
2     [Art. 1.880, Art. 1.860, Art. 1.860 Parágrafo ...
3     [Art. 1.860, Art. 1.799, Art. 1.900, Art. 1.80...
4            [Art. 1.723, Art. 1.723, § 1º, Art. 1.521]
5                            [Art. 276 do Código Civil]
6                        [Art. 1.601 do CC, CF de 1988]
7                            [Art. 270 do Código Civil]
8                     [Art. 1º da Resolução nº 35/2007]
9                       [Art. 1.813, Art. 1.813, § 1 o]
10    [Art. 1.860, Art. 1.858, Art. 1.786, Art. 1.84...
11                                         [Art. 1.867]
12                                         [Art. 1.864]
13    [Art. 1.844, Art. 1.836, Art. 1.830, Art. 1.83...
14                                   [Art. 1.785 do CC]
15    [Art. 1.798 do Código Civil, Art. 1.799 do Cód...
16    [Art. 1.696 do Código Civil, Art. 1.698 do Cód...
17                                         [Art.

In [ ]:
completed_artigos = []
for idx, commented_article in enumerate(df['artigos_nos_comentarios']):
    row_completed = []
    for article in commented_article:
        if 'Resolução' in article or 'ECA' in article or 'Estatuto' in article or 'CPC' in article:
            continue
        article = ('Art. ' + ''.join(article.split('Art.')[1:]).strip().split(' ')[0]).strip()
        if article == 'Art.':
            continue
        artigo_completo = get_article(article)
        if artigo_completo:
            row_completed.append(artigo_completo)
    completed_artigos.append(row_completed)

In [ ]:
df['artigos_completos'] = completed_artigos
df[['artigos_nos_comentarios',	'artigos_completos']]

,artigos_nos_comentarios,artigos_completos
0,"[Art. 1.826, Art. 1.214, Art. 1.214, Parágrafo...",[Art. 1.826. O possuidor da herança está obrig...
1,"[Art. 1.867 do Código Civil, Art. 1.868 do Cód...",[Art. 1.867. Ao cego só se permite o testament...
2,"[Art. 1.880, Art. 1.860, Art. 1.860 Parágrafo ...",[Art. 1.880. O testamento particular pode ser ...
3,"[Art. 1.860, Art. 1.799, Art. 1.900, Art. 1.80...","[Art. 1.860. Além dos incapazes, não podem tes..."
4,"[Art. 1.723, Art. 1.723, § 1º, Art. 1.521]",[Art. 1.723. É reconhecida como entidade famil...
5,[Art. 276 do Código Civil],[Art. 276. Se um dos devedores solidários fale...
6,"[Art. 1.601 do CC, CF de 1988]",[Art. 1.601. Cabe ao marido o direito de conte...
7,[Art. 270 do Código Civil],[Art. 270. Se um dos credores solidários falec...
8,[Art. 1º da Resolução nº 35/2007],[]
9,"[Art. 1.813, Art. 1.813, § 1 o]",[Art. 1.813. Quando o herdeiro prejudicar os s...


In [ ]:
df.to_csv("dataset_concurso_artigos_completos.csv", index=False)

## Configurando LLM

Para cada artigo comentado pergunta se ele ajuda a validar se as alternativas são verdadeiras ou falsas

In [104]:
class COMMENT_STATEMENT_PAIR_JSON(BaseModel):
    ajuda_a_responder: str = Field(description="True or False")
    justificativa: str = Field(description="Por que o artigo ajuda ou não a responder a questão")
    alternativas_relacionadas: List[str] = Field(description="Alternativas relacionadas ao artigo (se existir)")
    assertivas_relacionadas: List[str] = Field(description="Assertivas relacionadas ao artigo (se existir)")

output_parser = JsonOutputParser(pydantic_object=COMMENT_STATEMENT_PAIR_JSON)

In [107]:
prompt = """
Você é um estudante dedicado de direito do Código Civil Brasileiro.
Você quer saber se o artigo que você está estudando ajuda a responder algumas questões.
Se o artigo ajudar a responder, você quer saber com precisão quais alternativas ou quais assertivas
podem ser avaliadas usando o artigo como base.

artigo: {artigo}
enunciado: {enunciado}
assertivas: {assertivas}
alternativas: {alternativas}

Instruções de saída: {format_instructions}
""".strip()

prompt_template = PromptTemplate(
    input_variables=["artigo", "enunciado", "assertivas", "alternativas"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [109]:
def call_valid_article_statements(sample):
    sample_output = []
    if sample["artigos_completos"]:
        for artigo in sample["artigos_completos"]:
            output = chain.invoke({
                "artigo": artigo,
                "enunciado": sample["enunciado"],
                "assertivas": sample["assertivas"],
                "alternativas": sample["alternativas"],
            })

    else:
        output = chain.invoke({
                "artigo": 'Não encontrei artigos no Código Civil para responder essa questão',
                "enunciado": sample["enunciado"],
                "assertivas": sample["assertivas"],
                "alternativas": sample["alternativas"],
            })
    sample_output.append(output)
    return output

# test
call_valid_article_statements(df.iloc[0])

{'ajuda_a_responder': 'Sim',
 'justificativa': 'O artigo 206 do Código Civil Brasileiro ajuda a responder a questão pois trata da prescrição de pretensões, incluindo a prescrição de pretensões relativas a direitos reais, como a propriedade do sítio herdado por Diego.',
 'alternativas_relacionadas': ['B', 'E'],
 'assertivas_relacionadas': []}

## Loop

In [112]:
list_of_outputs = loop_fn(df, call_valid_article_statements)

  3%|▎         | 1/39 [01:10<44:20, 70.00s/it]

{'ajuda_a_responder': 'Sim', 'justificativa': 'O artigo 206 do Código Civil Brasileiro ajuda a responder a questão pois trata da prescrição de pretensões, incluindo a prescrição de pretensões relativas a direitos reais, como a propriedade do sítio herdado por Diego.', 'alternativas_relacionadas': ['B', 'E'], 'assertivas_relacionadas': []}


  5%|▌         | 2/39 [02:07<38:40, 62.73s/it]

{'ajuda_a_responder': 'Sim', 'justificativa': 'O artigo 1.880 do Código Civil Brasileiro ajuda a responder à questão pois trata da possibilidade de um testamento particular ser escrito em língua estrangeira, desde que as testemunhas o compreendam.', 'alternativas_relacionadas': ['B) I e IV.', 'C) II e IV.', 'D) III e IV.'], 'assertivas_relacionadas': ['IV. O testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a compreendam.']}


  8%|▊         | 3/39 [03:37<45:11, 75.32s/it]

...

{'ajuda_a_responder': 'Sim', 'justificativa': 'O artigo 1.857, § 1º do Código Civil Brasileiro estabelece que a legítima dos herdeiros necessários não pode ser incluída no testamento.', 'alternativas_relacionadas': ['C) Certo'], 'assertivas_relacionadas': ['A legítima dos herdeiros necessários não poderá ser incluída no testamento.']}


 97%|█████████▋| 38/39 [20:46<00:25, 25.72s/it]

{'ajuda_a_responder': 'Sim', 'justificativa': 'O artigo 1.822 do Código Civil Brasileiro estabelece que, decorridos cinco anos da abertura da sucessão, os bens arrecadados passarão ao domínio do Município ou do Distrito Federal, ou à União, se localizados em território federal. Isso significa que a petição de herança de Renato não está prescrita, pois o prazo de cinco anos não se consumou.', 'alternativas_relacionadas': ['A', 'E'], 'assertivas_relacionadas': []}


100%|██████████| 39/39 [21:00<00:00, 32.33s/it]

{'ajuda_a_responder': 'C) Certo', 'justificativa': 'O artigo 1.799 do Código Civil Brasileiro ajuda a responder à questão pois estabelece que os filhos, ainda não concebidos, de pessoas indicadas pelo testador, desde que vivas estas ao abrir-se a sucessão, podem ser chamados a suceder.', 'alternativas_relacionadas': [], 'assertivas_relacionadas': ['Ana, esposa de João, pode ser chamada a suceder, pois estava grávida quando João morreu.']}


In [115]:
df_article_relation = pd.DataFrame(list_of_outputs)
df_article_relation.to_csv("df_article_relation.csv", index=False)
!cp df_article_relation.csv /content/drive/MyDrive/

In [119]:
for alternativas, assertivas in zip(df_article_relation['alternativas_relacionadas'], df_article_relation['assertivas_relacionadas']):
    print(alternativas, assertivas)


['B', 'E'] []
['B) I e IV.', 'C) II e IV.', 'D) III e IV.'] ['IV. O testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a compreendam.']
['E) O testamento pode ser escrito em língua estrangeira desde que as testemunhas a compreendam.'] []
['C', 'D', 'E'] []
['A', 'D'] []
['C) Certo'] []
['C', 'E'] []
['C) Certo'] []
[] []
['E) deverá se habilitar no inventário dos bens de Judith para receber seu crédito.'] []
['A) O menor de dezoito anos não pode fazer o seu testamento.'] []
['A) necessariamente a forma pública. Nesse caso, o teor do documento deve ser lido para Marcelo em voz alta pelo tabelião e por uma das testemunhas.'] []
['B', 'C', 'D'] []
['E'] []
['C) Certo'] []
['E) Errado'] []
['B', 'E'] []
['C) Abandono involuntário do lar conjugal, durante um ano contínuo.'] []
['B', 'D'] []
['A', 'B', 'C'] []
['E'] []
['B) Apenas estão corretas as assertivas I e II.'] ['II. É ineficaz a cessão pelo coerdeiro de qualquer bem do acervo hereditário consid

In [125]:
df.iloc[[8, -4]]

,pergunta,resposta,comentarios,numero_na_captura,id,area,ano,banca,prova,duvidou_do_gabarito,enunciado,assertivas,alternativas,classificacao,artigos_nos_comentarios,artigos_completos
8,"Fernando, viúvo, faleceu em Brasília, seu últi...",E,art. 1º da Resolução nº 35/2007:\nArt. 1' Para...,10,Q3071800,Direito Civil Direito das Sucessões (Art. 1.7...,2023,Fundação Getúlio Vargas - FGV,FGV - Câmara dos Deputados - Analista Legisla...,False,"Fernando, viúvo, faleceu em Brasília, seu últi...",[],"['A) em Brasília.', 'B) no Rio de Janeiro.', '...",1,[Art. 1º da Resolução nº 35/2007],[]
35,Cada um dos itens a seguir apresenta uma situa...,D,GABARITO LETRA D\n(I) CORRETO.\nEnunciado 632 ...,37,Q2719195,Direito Civil Direito de Família (Art. 1.511 ...,2023,Centro de Seleção e de Promoção de Eventos Un...,CESPE/CEBRASPE - TJDFT - Juiz Substituto - 2023,False,Cada um dos itens a seguir apresenta uma situa...,"['I', 'II', 'III', 'IV']","['A) I e II', 'B) I e III', 'C) II e IV', 'D) ...",0,"[Art. 1.593 do Código Civil, Enunciado 632 do ...","[Art. 1.593. O parentesco é natural ou civil, ..."


Após análise optou-se por remover os indices 8 e -4 por não apresentarem resposta no código civil. No caso do -4 apenas uma das alternativas pode ser respondida pelo código civil. Ambas os modelos conseguem responder somente com seu conhecimento.

In [127]:
df = df.drop([8, len(df)-4])
df.reset_index(drop=True, inplace=True)
df.head()

,pergunta,resposta,comentarios,numero_na_captura,id,area,ano,banca,prova,duvidou_do_gabarito,enunciado,assertivas,alternativas,classificacao,artigos_nos_comentarios,artigos_completos
0,"Diego, em razão do falecimento do seu pai Eucl...",C,CC\nArt. 1.826. O possuidor da herança está ob...,1,Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...,True,"Diego, em razão do falecimento do seu pai Eucl...",[],"['A) todos os frutos já percebidos, bem como a...",1,"[Art. 1.826, Art. 1.214, Art. 1.214, Parágrafo...",[Art. 1.826. O possuidor da herança está obrig...
1,A sucessão testamentária é um componente impor...,B,I -\nArt. 1.867. Ao cego só se permite o testa...,2,Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,2024,Instituto Consulplan,Instituto Consulplan - DPE PR - Analista - Ár...,False,A sucessão testamentária é um componente impor...,['I. Ao cego só se permite o testamento públic...,"['A) I e II.', 'B) I e IV.', 'C) II e IV.', 'D...",0,"[Art. 1.867 do Código Civil, Art. 1.868 do Cód...",[Art. 1.867. Ao cego só se permite o testament...
2,"Matheus, filho de um milionário, aos dezesseis...",E,A resposta da questão está no Art. 1.880. O te...,3,Q3251846,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RO - Promotor de Justiça Substit...,False,"Matheus, filho de um milionário, aos dezesseis...",[],['A) O codicilo só é válido se não houver test...,1,"[Art. 1.880, Art. 1.860, Art. 1.860 Parágrafo ...",[Art. 1.880. O testamento particular pode ser ...
3,"Mariana, solteira, não convivente em união est...",C,"Art. 1.860. Além dos incapazes, não podem test...",4,Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação Getúlio Vargas - FGV,FGV - MPE GO - Promotor de Justiça Substituto...,True,"Mariana, solteira, não convivente em união est...",[],"['A) O testamento de Mariana, em razão da sua ...",1,"[Art. 1.860, Art. 1.799, Art. 1.900, Art. 1.80...","[Art. 1.860. Além dos incapazes, não podem tes..."
4,Rogério é casado com Cláudio e ambos são pais ...,C,Gabarito C.\n\nA preexistência de casamento ou...,6,Q3192677,Direito Civil Direito de Família (Art. 1.511 ...,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RJ - Promotor de Justiça Substit...,True,Rogério é casado com Cláudio e ambos são pais ...,[],['A) é possível a configuração da união estáve...,1,"[Art. 1.723, Art. 1.723, § 1º, Art. 1.521]",[Art. 1.723. É reconhecida como entidade famil...


In [143]:
len(df)

37

In [144]:
df.to_csv("dataset_concurso_limpo_segmentado.csv", index=False)

# RAG

## Criando Documentos e Carregando Embeddings

In [8]:
documents = artigos

In [9]:
with open("/content/codigocivil_embeddings_object.pkl", 'rb') as f:
    all_embeddings = pickle.load(f)

embeddings = [group['mean_embedding'] for group in all_embeddings]
embeddings = np.vstack(embeddings)

## Carregando Buscador

In [10]:
# Carregar o modelo de embeddings da Hugging Face
hf_embeddings = HuggingFaceEmbeddings(model_name="rufimelo/Legal-BERTimbau-sts-large-ma-v3")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.21k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/678k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


In [11]:
# Criar uma lista de objetos Document
docs = [Document(page_content='\n'.join(article_group)) for article_group in documents]
# Criar um InMemoryDocstore
docstore = InMemoryDocstore(dict(enumerate(docs)))

# Mapear índices para documentos
index_to_docstore_id = {i: i for i in range(len(docs))}

# Criar um índice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension) # Euclidian distance
index.add(embeddings)

# Criar o vetor FAISS a partir dos embeddings carregados e incluir a função de embeddings
vectorstore = FAISS(
    embedding_function=hf_embeddings,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

# Configurar o retriever para buscar os documentos relevantes
retriever = vectorstore.as_retriever()

In [152]:
pretty_print_docs(retriever.invoke('O que é Casamento?'))

Document 1:

Art. 1.723. É reconhecida como entidade familiar a união estável entre o homem e a mulher, configurada na convivência pública, contínua e duradoura e estabelecida com o objetivo de constituição de família.

§ 1 o A união estável não se constituirá se ocorrerem os impedimentos do art. 1.521 ; não se aplicando a incidência do inciso VI no caso de a pessoa casada se achar separada de fato ou judicialmente.

§ 2 o As causas suspensivas do art. 1.523 não impedirão a caracterização da união estável.

  
Art. 1.724. As relações pessoais entre os companheiros obedecerão aos deveres de lealdade, respeito e assistência, e de guarda, sustento e educação dos filhos.

  
Art. 1.725. Na união estável, salvo contrato escrito entre os companheiros, aplica-se às relações patrimoniais, no que couber, o regime da comunhão parcial de bens.

  
Art. 1.726. A união estável poderá converter-se em casamento, mediante pedido dos companheiros ao juiz e assento no Registro Civil.

  
Art. 1.727. As 

In [12]:
compressor = LLMChainExtractor.from_llm(llama3_70b_temp0)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)


## Preparando Alternativas e Assertivas

In [236]:
def convert_statments_to_list(strlist, seps=['I', 'II', 'III', 'IV', 'V']):
    if strlist == '[]':
        return []
    strlist = strlist[1:-1] # remove []
    strlist = strlist.replace("'", "") # remove '
    liststr = []
    old_stat = None
    for i, sep in enumerate(seps[:-1]):
        if sep not in strlist:
            continue
        stat = sep + ' ' + sep.join(strlist.split(sep)[1:]).strip().split(seps[i+1])[0].strip()
        if stat == old_stat:
            continue
        old_stat = stat
        if stat == 'C) Certo, E) Errado':
            stat = 'C) Certo'
        liststr.append(stat)
        print(stat)
    if liststr[-1] == liststr[-2]:
        liststr.pop(-1)
    return liststr

In [237]:
df = pd.read_csv("dataset_concurso_limpo_segmentado.csv")
assertivas_list = []
for alt in df['assertivas']:
    assertivas_list.append(convert_statments_to_list(alt))
df['assertivas'] = assertivas_list

I . Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.,
II . O testamento cerrado deve ser escrito mecanicamente, desde que seu subscritor numere e autentique, com a sua assinatura, todas as páginas.,
III . Em circunstâncias excepcionais declaradas na cédula, o testamento particular de próprio punho e assinado pelo testador, sem testemunhas, deverá ser confirmado, a critério do juiz.,
IV . O testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a compreendam.
I . O coerdeiro não poderá ceder onerosamente a sua quota hereditária à pessoa estranha à sucessão se outro coerdeiro a quiser adquirir, pelo mesmo valor.,
II . É ineficaz a cessão pelo coerdeiro de qualquer bem do acervo hereditário considerado singularmente.,
III . Realizada a cessão de direitos 

In [238]:
alternativas_list = []
for alt in df['alternativas']:
    alt = convert_statments_to_list(alt, seps=['A)', 'B)', 'C)', 'D)', 'E)', 'F)'])
    alternativas_list.append(alt)
df['alternativas'] = alternativas_list

A) todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias necessárias e úteis.,
B) todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção.,
C) todos os frutos percebidos até a citação, bem como o direito à indenização das benfeitorias, mas só pode exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.,
D) todos os frutos percebidos até a citação, bem como o direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção.,
E) todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de J

In [239]:
df.columns

Index(['pergunta', 'resposta', 'comentarios', 'numero_na_captura', 'id',
       'area', 'ano', 'banca', 'prova', 'duvidou_do_gabarito', 'enunciado',
       'assertivas', 'alternativas', 'classificacao',
       'artigos_nos_comentarios', 'artigos_completos'],
      dtype='object')

In [240]:
iteration_list = []
for idx, row in df.iterrows():
    if row['classificacao'] == 0:
        check_truthfulness_obj = [assertiva for assertiva in row['assertivas']]
    elif row['classificacao'] == 1:
        check_truthfulness_obj = [alternativa for alternativa in row['alternativas']]
    else:
        check_truthfulness_obj = '\n'.join(row['alternativas'])
    question_context = row['enunciado']
    articles = row["artigos_completos"]
    for c in check_truthfulness_obj:
        iteration_list.append({
            "contexto": question_context,
            "texto_para_checagem": c,
            "artigos": articles,})

iteration_df = pd.DataFrame(iteration_list)
iteration_df.head()

,contexto,texto_para_checagem,artigos
0,"Diego, em razão do falecimento do seu pai Eucl...","A) todos os frutos já percebidos, bem como aqu...",['Art. 1.826. O possuidor da herança está obri...
1,"Diego, em razão do falecimento do seu pai Eucl...","B) todos os frutos já percebidos, bem como aqu...",['Art. 1.826. O possuidor da herança está obri...
2,"Diego, em razão do falecimento do seu pai Eucl...","C) todos os frutos percebidos até a citação, b...",['Art. 1.826. O possuidor da herança está obri...
3,"Diego, em razão do falecimento do seu pai Eucl...","D) todos os frutos percebidos até a citação, b...",['Art. 1.826. O possuidor da herança está obri...
4,"Diego, em razão do falecimento do seu pai Eucl...","E) todos os frutos já percebidos, bem como aqu...",['Art. 1.826. O possuidor da herança está obri...


In [241]:
len(iteration_df)

607

## Fazendo a busca

In [249]:
for idx, texto in enumerate(iteration_df['texto_para_checagem']):
    print(texto)
    result = compression_retriever.invoke(texto)
    for doc in result:
        print(doc.page_content)
    print('\n--------------', iteration_df.iloc[idx]['artigos'])

    break

A) todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias necessárias e úteis.,
Art. 964. Têm privilégio especial:

III - sobre a coisa beneficiada, o credor por benfeitorias necessárias ou úteis;
Art. 1.396. Salvo direito adquirido por outrem, o usufrutuário faz seus os frutos naturais, pendentes ao começar o usufruto, sem encargo de pagar as despesas de produção.

Art. 1.398. Os frutos civis, vencidos na data inicial do usufruto, pertencem ao proprietário, e ao usufrutuário os vencidos na data em que cessa o usufruto.

-------------- ['Art. 1.826. O possuidor da herança está obrigado à restituição dos bens do acervo, fixando-se-lhe a responsabilidade segundo a sua posse, observado o disposto nos arts. 1.214 a 1.222 .Parágrafo único. A partir da citação, a responsabilidade do possuidor se há de aferir pelas regras concernentes à posse de má-f

In [250]:
iteration_df.to_csv("iteration_df.csv", index=False)

# Criando Termos de Busca

In [289]:
class FakeArticle(BaseModel):
    artigo_inventado: str = Field(description="Artigo criado artificialmente")
    topicos_para_busca: List[str] = Field(description="Topicos para busca no Código Civil")
    artigos_candidatos: List[str] = Field(description="Artigos candidatos para suporte a resolução da questão")
    frase_assunto_da_questão: str = Field(description="Frase do assunto da questão")

output_parser = JsonOutputParser(pydantic_object=FakeArticle)

In [292]:
prompt = """
Você é um legislador e ávido conhecedor do Código Civil Brasileiro.
Use seus conhecimentos em leis e códigos para a partir de uma questão de concurso,
criar um artigo que possa responder os itens da questão.

Além disso, forneça:

- os tópicos para uma busca no Código Civil.
- possíveis artigos do Código Civil que respondem a questão.
- uma frase do assunto da questão buscando responder a questão.

questão: {questão}

Instruções de saída: {format_instructions}
""".strip()

prompt_template = PromptTemplate(
    input_variables=["questão"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [293]:
def call_hipoteses_generator(sample):
    output = chain.invoke({"questão": sample['pergunta'],})
    return output

# test
out = call_hipoteses_generator(df.iloc[0])
out

{'artigo_inventado': 'Art. 1.815-A. O herdeiro aparente e de boa-fé tem direito a todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos do verdadeiro herdeiro, e também tem direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção.',
 'topicos_para_busca': ['Herdeiro aparente',
  'Boa-fé',
  'Frutos',
  'Benfeitorias',
  'Direito de retenção'],
 'artigos_candidatos': ['Art. 1.815',
  'Art. 1.816',
  'Art. 1.817',
  'Art. 1.818'],
 'frase_assunto_da_questao': 'Direitos do herdeiro aparente e de boa-fé em caso de nulidade de inventário'}

In [269]:
df.iloc[0]['artigos_nos_comentarios']

"['Art. 1.826', 'Art. 1.214', 'Art. 1.214', 'Parágrafo único', 'Art. 1.219', 'Art. 1.973', 'Art. 1.974', 'Art. 1.975', 'Art. 205', 'Art. 206']"

In [286]:
texto = retriever.invoke(out['artigo_inventado'])
print_only_artID(edit_docs(texto))

['Art. 955', 'Art. 956', 'Art. 957', 'Art. 958', 'Art. 959', 'Art. 960', 'Art. 961', 'Art. 962', 'Art. 963', 'Art. 964', 'Art. 965', 'Art. 1.824', 'Art. 1.825', 'Art. 1.826', 'Art. 1.827', 'Art. 1.828', 'Art. 1.238', 'Art. 1.239', 'Art. 1.240', 'Art. 1.240-A', 'Art. 1.241', 'Art. 1.242', 'Art. 1.243', 'Art. 1.244', 'Art. 1.976', 'Art. 1.977', 'Art. 1.978', 'Art. 1.979', 'Art. 1.980', 'Art. 1.981', 'Art. 1.982', 'Art. 1.983', 'Art. 1.984', 'Art. 1.985', 'Art. 1.986', 'Art. 1.987', 'Art. 1.988', 'Art. 1.989', 'Art. 1.990']


In [287]:
texto = retriever.invoke(out['topicos_para_busca'][0])
print_only_artID(edit_docs(texto))

['Art. 1.912', 'Art. 1.913', 'Art. 1.914', 'Art. 1.915', 'Art. 1.916', 'Art. 1.917', 'Art. 1.918', 'Art. 1.919', 'Art. 1.920', 'Art. 1.921', 'Art. 1.922', 'Art. 11', 'Art. 12', 'Art. 13', 'Art. 14', 'Art. 15', 'Art. 16', 'Art. 17', 'Art. 18', 'Art. 19', 'Art. 20', 'Art. 21', 'Art. 1.851', 'Art. 1.852', 'Art. 1.853', 'Art. 1.854', 'Art. 1.855', 'Art. 1.856', 'Art. 1º Toda pessoa é capaz de direitos e deveres na ordem civil.', 'Art. 2º A personalidade civil da pessoa começa do nascimento com vida; mas a lei põe a salvo, desde a concepção, os direitos do nascituro.', 'Art. 3 o São absolutamente incapazes de exercer pessoalmente os atos da vida civil:I - os menores de dezesseis anos;II - os que, por enfermidade ou deficiência mental, não tiverem o necessário discernimento para a prática desses atos;III - os que, mesmo por causa transitória, não puderem exprimir sua vontade.', 'Art. 3º São absolutamente incapazes de exercer pessoalmente os atos da vida civil os menores de 16 (dezesseis) ano

In [288]:
texto = retriever.invoke(out['topicos_para_busca'][1])
print_only_artID(edit_docs(texto))

['Art. 1.210', 'Art. 1.211', 'Art. 1.212', 'Art. 1.213', 'Art. 1.214', 'Art. 1.215', 'Art. 1.216', 'Art. 1.217', 'Art. 1.218', 'Art. 1.219', 'Art. 1.220', 'Art. 1.221', 'Art. 1.222', 'Art. 1.196', 'Art. 1.197', 'Art. 1.198', 'Art. 1.199', 'Art. 1.200', 'Art. 1.201', 'Art. 1.202', 'Art. 1.203', 'Art. 104', 'Art. 105', 'Art. 106', 'Art. 107', 'Art. 108', 'Art. 109', 'Art. 110', 'Art. 111', 'Art. 112', 'Art. 113', 'Art. 114', 'Art. 92', 'Art. 93', 'Art. 94', 'Art. 95', 'Art. 96', 'Art. 97']


In [294]:
texto = retriever.invoke(out['frase_assunto_da_questao'])
print_only_artID(edit_docs(texto))

['Art. 1.824', 'Art. 1.825', 'Art. 1.826', 'Art. 1.827', 'Art. 1.828', 'Art. 1.814', 'Art. 1.815', 'Art. 1.815-A', 'Art. 1.816', 'Art. 1.817', 'Art. 1.818', 'Art. 1.791', 'Art. 1.792', 'Art. 1.793', 'Art. 1.794', 'Art. 1.795', 'Art. 1.796', 'Art. 1.797', 'Art. 1.976', 'Art. 1.977', 'Art. 1.978', 'Art. 1.979', 'Art. 1.980', 'Art. 1.981', 'Art. 1.982', 'Art. 1.983', 'Art. 1.984', 'Art. 1.985', 'Art. 1.986', 'Art. 1.987', 'Art. 1.988', 'Art. 1.989', 'Art. 1.990']


In [298]:
texto = compression_retriever.invoke(out['frase_assunto_da_questao'])
print_only_artID(edit_docs(texto))

['Art. 1.827']


In [296]:
get_article('Art. 1.827')

'Art. 1.827. O herdeiro pode demandar os bens da herança, mesmo em poder de terceiros, sem prejuízo da responsabilidade do possuidor originário pelo valor dos bens alienados.Parágrafo único. São eficazes as alienações feitas, a título oneroso, pelo herdeiro aparente a terceiro de boa-fé.'

In [301]:
texto = retriever.invoke(' '.join(out['topicos_para_busca']))
print_only_artID(edit_docs(texto))


['Art. 92', 'Art. 93', 'Art. 94', 'Art. 95', 'Art. 96', 'Art. 97', 'Art. 1.210', 'Art. 1.211', 'Art. 1.212', 'Art. 1.213', 'Art. 1.214', 'Art. 1.215', 'Art. 1.216', 'Art. 1.217', 'Art. 1.218', 'Art. 1.219', 'Art. 1.220', 'Art. 1.221', 'Art. 1.222', 'Art. 955', 'Art. 956', 'Art. 957', 'Art. 958', 'Art. 959', 'Art. 960', 'Art. 961', 'Art. 962', 'Art. 963', 'Art. 964', 'Art. 965', 'Art. 158', 'Art. 159', 'Art. 160', 'Art. 161', 'Art. 162', 'Art. 163', 'Art. 164', 'Art. 165']


In [303]:
texto = compression_retriever.invoke(' '.join(out['topicos_para_busca']))
print_only_artID(edit_docs(texto))

['Art. 96', 'Art. 1.214', 'Art. 1.219', 'Art. 964', 'Art. 164']


In [306]:
texto = compression_retriever.invoke(out['artigo_inventado'])
edit_docs(texto)

'Document 1:\n\nArt. 1.827. Parágrafo único. São eficazes as alienações feitas, a título oneroso, pelo herdeiro aparente a terceiro de boa-fé.\n\nArt. 1.828. O herdeiro aparente, que de boa-fé houver pago um legado, não está obrigado a prestar o equivalente ao verdadeiro sucessor, ressalvado a este o direito de proceder contra quem o recebeu.\n----------------------------------------------------------------------------------------------------\nDocument 2:\n\nArt. 1.815-A. O herdeiro aparente e de boa-fé tem direito a todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos do verdadeiro herdeiro, e também tem direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção.'

## Loop

In [307]:
list_of_outputs = loop_fn(df, call_hipoteses_generator)

  3%|▎         | 1/37 [00:01<00:59,  1.65s/it]

{'artigo_inventado': 'Art. 1.815-A. O herdeiro aparente e de boa-fé tem direito a todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos do verdadeiro herdeiro, e também tem direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção.', 'topicos_para_busca': ['Herdeiro aparente', 'Boa-fé', 'Frutos', 'Benfeitorias', 'Direito de retenção'], 'artigos_candidatos': ['Art. 1.815', 'Art. 1.816', 'Art. 1.817', 'Art. 1.818'], 'frase_assunto_da_questao': 'Direitos do herdeiro aparente e de boa-fé em caso de nulidade de inventário'}


  5%|▌         | 2/37 [00:03<00:55,  1.58s/it]

{'artigo_inventado': 'Art. 1.866-A. A sucessão testamentária é regulada pelas disposições do presente Código, garantindo a vontade do testador e a segurança jurídica na transmissão de bens após a morte.', 'topicos_para_busca': ['Sucessão testamentária', 'Testamento', 'Formas de testamento', 'Requisitos de validade do testamento'], 'artigos_candidatos': ['Art. 1.862', 'Art. 1.863', 'Art. 1.864', 'Art. 1.865'], 'frase_assunto_da_questao': 'A sucessão testamentária é um componente importante do planejamento patrimonial e permite que as pessoas tenham controle sobre o destino de seus bens após a morte.'}


  8%|▊         | 3/37 [00:04<00:55,  1.63s/it]

...

{'artigo_inventado': 'Art. 1.848. A legítima dos herdeiros necessários é irrenunciável e imprescritível, não podendo ser incluída no testamento.', 'topicos_para_busca': ['Herdeiros necessários', 'Legítima', 'Testamento'], 'artigos_candidatos': ['Art. 1.845', 'Art. 1.846', 'Art. 1.848'], 'frase_assunto_da_questao': 'A legítima dos herdeiros necessários é protegida por lei e não pode ser afetada pela vontade do testador.'}


 97%|█████████▋| 36/37 [06:03<00:12, 12.06s/it]

{'artigo_inventado': 'Art. 1.844-A. A petição de herança em decorrência de investigação de paternidade post mortem não está sujeita a prescrição, desde que a ação seja ajuizada dentro do prazo de 10 (dez) anos contados da abertura da sucessão.', 'topicos_para_busca': ['Paternidade', 'Investigação de paternidade', 'Paternidade post mortem', 'Herança', 'Prescrição'], 'artigos_candidatos': ['Art. 1.835', 'Art. 1.836', 'Art. 1.837', 'Art. 1.838', 'Art. 1.839', 'Art. 1.844'], 'frase_assunto_da_questao': 'A petição de herança em decorrência de investigação de paternidade post mortem não está sujeita a prescrição, desde que a ação seja ajuizada dentro do prazo de 10 (dez) anos contados da abertura da sucessão.'}


100%|██████████| 37/37 [06:14<00:00, 10.11s/it]

{'artigo_inventado': 'Art. 1.798-A. O nascituro tem legitimidade para suceder, desde que o pai tenha reconhecido a paternidade ou que a mãe tenha declarado a gravidez no momento da abertura da sucessão.', 'topicos_para_busca': ['Direito das Sucessões', 'Nascituro', 'Legitimidade para suceder', 'Reconhecimento de paternidade'], 'artigos_candidatos': ['Art. 1.798', 'Art. 1.799', 'Art. 1.800'], 'frase_assunto_da_questao': 'O nascituro tem legitimidade para suceder desde que o pai tenha reconhecido a paternidade ou que a mãe tenha declarado a gravidez no momento da abertura da sucessão.'}


In [308]:
df_termos_busca = pd.DataFrame(list_of_outputs)
df_termos_busca.to_csv("df_termos_busca.csv", index=False)
!cp df_termos_busca.csv /content/drive/MyDrive/

In [314]:
df = pd.concat([df, df_termos_busca], axis=1)
df.head()

,pergunta,resposta,comentarios,numero_na_captura,id,area,ano,banca,prova,duvidou_do_gabarito,...,artigos_nos_comentarios,artigos_completos,artigo_inventado,topicos_para_busca,artigos_candidatos,frase_assunto_da_questao,artigo_inventado,topicos_para_busca,artigos_candidatos,frase_assunto_da_questao
0,"Diego, em razão do falecimento do seu pai Eucl...",C,CC\nArt. 1.826. O possuidor da herança está ob...,1,Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...,True,...,"['Art. 1.826', 'Art. 1.214', 'Art. 1.214', 'Pa...",['Art. 1.826. O possuidor da herança está obri...,"[, herdeir, aparent, , d, boa-f, te, direit, ,...","['Herdeiro aparente', 'Boa-fé', 'Frutos', 'Ben...","['Art. 1.815', 'Art. 1.816', 'Art. 1.817', 'Ar...",Direitos do herdeiro aparente e de boa-fé em c...,Art. 1.815-A. O herdeiro aparente e de boa-fé ...,"[Herdeiro aparente, Boa-fé, Frutos, Benfeitori...","[Art. 1.815, Art. 1.816, Art. 1.817, Art. 1.818]",Direitos do herdeiro aparente e de boa-fé em c...
1,A sucessão testamentária é um componente impor...,B,I -\nArt. 1.867. Ao cego só se permite o testa...,2,Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,2024,Instituto Consulplan,Instituto Consulplan - DPE PR - Analista - Ár...,False,...,"['Art. 1.867 do Código Civil', 'Art. 1.868 do ...",['Art. 1.867. Ao cego só se permite o testamen...,"[, sucessã, testamentári, , regulad, pela, dis...","['Sucessão testamentária', 'Testamento', 'Form...","['Art. 1.862', 'Art. 1.863', 'Art. 1.864', 'Ar...",A sucessão testamentária é um componente impor...,Art. 1.866-A. A sucessão testamentária é regul...,"[Sucessão testamentária, Testamento, Formas de...","[Art. 1.862, Art. 1.863, Art. 1.864, Art. 1.865]",A sucessão testamentária é um componente impor...
2,"Matheus, filho de um milionário, aos dezesseis...",E,A resposta da questão está no Art. 1.880. O te...,3,Q3251846,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RO - Promotor de Justiça Substit...,False,...,"['Art. 1.880', 'Art. 1.860', 'Art. 1.860 Parág...",['Art. 1.880. O testamento particular pode ser...,"[, testament, pod, se, escrit, e, língu, estra...","['Testamento', 'Língua estrangeira', 'Alteraçã...","['Art. 1.857', 'Art. 1.858', 'Art. 1.862', 'Ar...",O testamento de menor pode ser válido e altera...,Art. 1.862-A. O testamento pode ser escrito em...,"[Testamento, Língua estrangeira, Alteração de ...","[Art. 1.857, Art. 1.858, Art. 1.862, Art. 1.864]",O testamento de menor pode ser válido e altera...
3,"Mariana, solteira, não convivente em união est...",C,"Art. 1.860. Além dos incapazes, não podem test...",4,Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação Getúlio Vargas - FGV,FGV - MPE GO - Promotor de Justiça Substituto...,True,...,"['Art. 1.860', 'Art. 1.799', 'Art. 1.900', 'Ar...","['Art. 1.860. Além dos incapazes, não podem te...","[, testament, particular, elaborad, po, pesso,...","['Testamento particular', 'Incapacidade relati...","['Art. 1.860 do Código Civil', 'Art. 1.857 do ...",A validade do testamento particular elaborado ...,"Art. 1.859-A. O testamento particular, elabora...","[Testamento particular, Incapacidade relativa,...","[Art. 1.860 do Código Civil, Art. 1.857 do Cód...",A validade do testamento particular elaborado ...
4,Rogério é casado com Cláudio e ambos são pais ...,C,Gabarito C.\n\nA preexistência de casamento ou...,6,Q3192677,Direito Civil Direito de Família (Art. 1.511 ...,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RJ - Promotor de Justiça Substit...,True,...,"['Art. 1.723', 'Art. 1.723', '§ 1º', 'Art. 1.5...",['Art. 1.723. É reconhecida como entidade fami...,"[, reconhecid, com, uniã, estáve, , relaçã, en...","['União Estável', 'Convivência Pública', 'Rela...","['Art. 1.723', 'Art. 1.726', 'Art. 1.727', 'Ar...",A configuração da união estável entre pessoas ...,Art. 1.723-A. É reco

In [310]:
df.to_csv("dataset_concurso_limpo_segmentado_termos_busca.csv", index=False)
!cp dataset_concurso_limpo_segmentado_termos_busca.csv /content/drive/MyDrive/

In [318]:
df = pd.read_csv("dataset_concurso_limpo_segmentado_termos_busca.csv")
df["artigo_inventado"] = df["artigo_inventado"].str.replace('.', '').str.strip().str.split(' ').str[2:].str.join(' ')
df.head()

,pergunta,resposta,comentarios,numero_na_captura,id,area,ano,banca,prova,duvidou_do_gabarito,enunciado,assertivas,alternativas,classificacao,artigos_nos_comentarios,artigos_completos,artigo_inventado,topicos_para_busca,artigos_candidatos,frase_assunto_da_questao
0,"Diego, em razão do falecimento do seu pai Eucl...",C,CC\nArt. 1.826. O possuidor da herança está ob...,1,Q3116412,Direito Civil Direito das Sucessões (Art. 1.7...,2024,Fundação Getúlio Vargas - FGV,FGV - Câmara de São Paulo - Procurador Legisl...,True,"Diego, em razão do falecimento do seu pai Eucl...",[],"['A) todos os frutos já percebidos, bem como a...",1,"['Art. 1.826', 'Art. 1.214', 'Art. 1.214', 'Pa...",['Art. 1.826. O possuidor da herança está obri...,O herdeiro aparente e de boa-fé tem direito a ...,"['Herdeiro aparente', 'Boa-fé', 'Frutos', 'Ben...","['Art. 1.815', 'Art. 1.816', 'Art. 1.817', 'Ar...",Direitos do herdeiro aparente e de boa-fé em c...
1,A sucessão testamentária é um componente impor...,B,I -\nArt. 1.867. Ao cego só se permite o testa...,2,Q3121863,Direito Civil Sucessão Testamentária (Art. 1....,2024,Instituto Consulplan,Instituto Consulplan - DPE PR - Analista - Ár...,False,A sucessão testamentária é um componente impor...,['I . Ao cego só se permite o testamento públi...,"['A) I e II.,', 'B) I e IV.,', 'C) II e IV.,',...",0,"['Art. 1.867 do Código Civil', 'Art. 1.868 do ...",['Art. 1.867. Ao cego só se permite o testamen...,A sucessão testamentária é regulada pelas disp...,"['Sucessão testamentária', 'Testamento', 'Form...","['Art. 1.862', 'Art. 1.863', 'Art. 1.864', 'Ar...",A sucessão testamentária é um componente impor...
2,"Matheus, filho de um milionário, aos dezesseis...",E,A resposta da questão está no Art. 1.880. O te...,3,Q3251846,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RO - Promotor de Justiça Substit...,False,"Matheus, filho de um milionário, aos dezesseis...",[],['A) O codicilo só é válido se não houver test...,1,"['Art. 1.880', 'Art. 1.860', 'Art. 1.860 Parág...",['Art. 1.880. O testamento particular pode ser...,O testamento pode ser escrito em língua estran...,"['Testamento', 'Língua estrangeira', 'Alteraçã...","['Art. 1.857', 'Art. 1.858', 'Art. 1.862', 'Ar...",O testamento de menor pode ser válido e altera...
3,"Mariana, solteira, não convivente em união est...",C,"Art. 1.860. Além dos incapazes, não podem test...",4,Q3158359,Direito Civil Sucessão Testamentária (Art. 1....,2024,Fundação Getúlio Vargas - FGV,FGV - MPE GO - Promotor de Justiça Substituto...,True,"Mariana, solteira, não convivente em união est...",[],"['A) O testamento de Mariana, em razão da sua ...",1,"['Art. 1.860', 'Art. 1.799', 'Art. 1.900', 'Ar...","['Art. 1.860. Além dos incapazes, não podem te...","O testamento particular, elaborado por pessoa ...","['Testamento particular', 'Incapacidade relati...","['Art. 1.860 do Código Civil', 'Art. 1.857 do ...",A validade do testamento particular elaborado ...
4,Rogério é casado com Cláudio e ambos são pais ...,C,Gabarito C.\n\nA preexistência de casamento ou...,6,Q3192677,Direito Civil Direito de Família (Art. 1.511 ...,2024,Fundação para o Vestibular da Universidade Es...,VUNESP - MPE RJ - Promotor de Justiça Substit...,True,Rogério é casado com Cláudio e ambos são pais ...,[],['A) é possível a configuração da união estáve...,1,"['Art. 1.723', 'Art. 1.723', '§ 1º', 'Art. 1.5...",['Art. 1.723. É reconhecida como entidade fami...,É reconhecida como união estável a relação ent...,"['União Estável', 'Convivência Pública', 'Rela...","['Art. 1.723', 'Art. 1.726', 'Art. 1.727', 'Ar...",A configuração da união estável entre pessoas ...


# LOOP das Buscas

## Artigo Inventado

In [13]:
df = pd.read_csv("dataset_concurso_limpo_segmentado_termos_busca.csv")

In [76]:
conhecimento_externo = []
for idx, texto in enumerate(df['artigo_inventado']):
    print(texto)
    result = edit_docs_clean(retriever.invoke(texto))
    print_only_artID(result)
    conhecimento_externo.append(result)

df['rag_art_inventado_full'] = conhecimento_externo

O herdeiro aparente e de boa-fé tem direito a todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos do verdadeiro herdeiro, e também tem direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção
['Art. 1.238', 'Art. 1.239', 'Art. 1.240', 'Art. 1.240-A', 'Art. 1.241', 'Art. 1.242', 'Art. 1.243', 'Art. 1.244', 'Art. 26', 'Art. 27', 'Art. 28', 'Art. 29', 'Art. 30', 'Art. 31', 'Art. 32', 'Art. 33', 'Art. 34', 'Art. 35', 'Art. 36', 'Art. 505', 'Art. 506', 'Art. 507', 'Art. 508', 'Art. 1.228', 'Art. 1.229', 'Art. 1.230', 'Art. 1.231', 'Art. 1.232']
A sucessão testamentária é regulada pelas disposições do presente Código, garantindo a vontade do testador e a segurança jurídica na transmissão de bens após a morte
['Art. 1.976', 'Art. 1.977', 'Art. 1.978', 'Art. 1.979', 'Art. 1.980', 'Art. 1.981', 'Art. 1.982', 'Art. 1.983', 'Art. 1.984', 'Art. 1.985', 'Art. 1.986'

In [18]:
conhecimento_externo = []
for idx, texto in enumerate(df['artigo_inventado']):
    print(texto)
    result = edit_docs(compression_retriever.invoke(texto))
    print_only_artID(result)
    conhecimento_externo.append(result)

df['rag_art_inventado_compress'] = conhecimento_externo

Art. 1.815-A. O herdeiro aparente e de boa-fé tem direito a todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos do verdadeiro herdeiro, e também tem direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção.
['Art. 1.827', 'Art. 1.828', 'Art. 1.815-A']
Art. 1.866-A. A sucessão testamentária é regulada pelas disposições do presente Código, garantindo a vontade do testador e a segurança jurídica na transmissão de bens após a morte.
[]
Art. 1.862-A. O testamento pode ser escrito em língua estrangeira, desde que as testemunhas o compreendam, e pode ser alterado a qualquer tempo, mediante manifestação de última vontade do testador, desde que seja lida e assinada na presença de três testemunhas.
['Art. 1.880', 'Art. 1.871', 'Art. 1.858']
Art. 1.859-A. O testamento particular, elaborado por pessoa relativamente incapaz em razão da idade, é válido se, ao tempo d

KeyboardInterrupt: 

In [19]:
df.to_csv("dataset_concurso_rag.csv", index=False)
!cp dataset_concurso_rag.csv /content/drive/MyDrive/

In [20]:
df["artigo_inventado"] = df["artigo_inventado"].str.replace('.', '').str.strip().str.split(' ').str[2:].str.join(' ')

In [78]:
conhecimento_externo = []
for idx, texto in enumerate(df['artigo_inventado']):
    print(texto)
    result = edit_docs_clean(retriever.invoke(texto))
    print_only_artID(result)
    conhecimento_externo.append(result)

df['rag_art_inventado_full_sem_art'] = conhecimento_externo

O herdeiro aparente e de boa-fé tem direito a todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos do verdadeiro herdeiro, e também tem direito à indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção
['Art. 1.238', 'Art. 1.239', 'Art. 1.240', 'Art. 1.240-A', 'Art. 1.241', 'Art. 1.242', 'Art. 1.243', 'Art. 1.244', 'Art. 26', 'Art. 27', 'Art. 28', 'Art. 29', 'Art. 30', 'Art. 31', 'Art. 32', 'Art. 33', 'Art. 34', 'Art. 35', 'Art. 36', 'Art. 505', 'Art. 506', 'Art. 507', 'Art. 508', 'Art. 1.228', 'Art. 1.229', 'Art. 1.230', 'Art. 1.231', 'Art. 1.232']
A sucessão testamentária é regulada pelas disposições do presente Código, garantindo a vontade do testador e a segurança jurídica na transmissão de bens após a morte
['Art. 1.976', 'Art. 1.977', 'Art. 1.978', 'Art. 1.979', 'Art. 1.980', 'Art. 1.981', 'Art. 1.982', 'Art. 1.983', 'Art. 1.984', 'Art. 1.985', 'Art. 1.986'

In [ ]:
conhecimento_externo = []
for idx, texto in enumerate(df['artigo_inventado']):
    print(texto)
    result = edit_docs(compression_retriever.invoke(texto))
    print_only_artID(result)
    conhecimento_externo.append(result)

df['rag_art_inventado_compress_sem_art'] = conhecimento_externo

In [22]:
df.to_csv("dataset_concurso_rag.csv", index=False)
!cp dataset_concurso_rag.csv /content/drive/MyDrive/

## Frase Assunto

In [79]:
conhecimento_externo = []
for idx, texto in enumerate(df['frase_assunto_da_questao']):
    print(texto)
    result = edit_docs_clean(retriever.invoke(texto))
    print_only_artID(result)
    conhecimento_externo.append(result)

df['rag_assunto_full'] = conhecimento_externo

Direitos do herdeiro aparente e de boa-fé em caso de nulidade de inventário
['Art. 1.824', 'Art. 1.825', 'Art. 1.826', 'Art. 1.827', 'Art. 1.828', 'Art. 1.814', 'Art. 1.815', 'Art. 1.815-A', 'Art. 1.816', 'Art. 1.817', 'Art. 1.818', 'Art. 1.791', 'Art. 1.792', 'Art. 1.793', 'Art. 1.794', 'Art. 1.795', 'Art. 1.796', 'Art. 1.797', 'Art. 1.976', 'Art. 1.977', 'Art. 1.978', 'Art. 1.979', 'Art. 1.980', 'Art. 1.981', 'Art. 1.982', 'Art. 1.983', 'Art. 1.984', 'Art. 1.985', 'Art. 1.986', 'Art. 1.987', 'Art. 1.988', 'Art. 1.989', 'Art. 1.990']
A sucessão testamentária é um componente importante do planejamento patrimonial e permite que as pessoas tenham controle sobre o destino de seus bens após a morte.
['Art. 26', 'Art. 27', 'Art. 28', 'Art. 29', 'Art. 30', 'Art. 31', 'Art. 32', 'Art. 33', 'Art. 34', 'Art. 35', 'Art. 36', 'Art. 1.912', 'Art. 1.913', 'Art. 1.914', 'Art. 1.915', 'Art. 1.916', 'Art. 1.917', 'Art. 1.918', 'Art. 1.919', 'Art. 1.920', 'Art. 1.921', 'Art. 1.922', 'Art. 1.923', 'Art.

In [ ]:
conhecimento_externo = []
for idx, texto in enumerate(df['frase_assunto_da_questao']):
    print(texto)
    result = edit_docs(compression_retriever.invoke(texto))
    print(result)
    print_only_artID(result)
    conhecimento_externo.append(result)

df['rag_assunto_compress'] = conhecimento_externo

In [24]:
df.to_csv("dataset_concurso_rag.csv", index=False)
!cp dataset_concurso_rag.csv /content/drive/MyDrive/

## Concatenação de tópicos

In [80]:
conhecimento_externo = []
for idx, texto in enumerate(df['topicos_para_busca']):
    texto = texto[1:-1].replace(',', '')
    print(texto)
    result = edit_docs_clean(retriever.invoke(texto))
    print_only_artID(result)
    conhecimento_externo.append(result)

df['rag_topicos_full'] = conhecimento_externo

'Herdeiro aparente' 'Boa-fé' 'Frutos' 'Benfeitorias' 'Direito de retenção'
['Art. 1.210', 'Art. 1.211', 'Art. 1.212', 'Art. 1.213', 'Art. 1.214', 'Art. 1.215', 'Art. 1.216', 'Art. 1.217', 'Art. 1.218', 'Art. 1.219', 'Art. 1.220', 'Art. 1.221', 'Art. 1.222', 'Art. 92', 'Art. 93', 'Art. 94', 'Art. 95', 'Art. 96', 'Art. 97', 'Art. 955', 'Art. 956', 'Art. 957', 'Art. 958', 'Art. 959', 'Art. 960', 'Art. 961', 'Art. 962', 'Art. 963', 'Art. 964', 'Art. 965', 'Art. 1.196', 'Art. 1.197', 'Art. 1.198', 'Art. 1.199', 'Art. 1.200', 'Art. 1.201', 'Art. 1.202', 'Art. 1.203']
'Sucessão testamentária' 'Testamento' 'Formas de testamento' 'Requisitos de validade do testamento'
['Art. 1.784', 'Art. 1.785', 'Art. 1.786', 'Art. 1.787', 'Art. 1.788', 'Art. 1.789', 'Art. 1.790', 'Art. 1.976', 'Art. 1.977', 'Art. 1.978', 'Art. 1.979', 'Art. 1.980', 'Art. 1.981', 'Art. 1.982', 'Art. 1.983', 'Art. 1.984', 'Art. 1.985', 'Art. 1.986', 'Art. 1.987', 'Art. 1.988', 'Art. 1.989', 'Art. 1.990', 'Art. 910', 'Art. 911',

In [ ]:
conhecimento_externo = []
for idx, texto in enumerate(df['topicos_para_busca']):
    texto = ' '.join(texto)
    print(texto)
    result = edit_docs(compression_retriever.invoke(texto))
    print(result)
    print_only_artID(result)
    conhecimento_externo.append(result)

df['rag_topicos_compress'] = conhecimento_externo

In [29]:
df.to_csv("dataset_concurso_rag.csv", index=False)
!cp dataset_concurso_rag.csv /content/drive/MyDrive/

## Possíveis Artigos

In [42]:
articles = []
for article in df['artigos_candidatos']:
    article = convert_strlist_to_liststr(article)
    article = list(set([' '.join(art.split(' ')[:2]) for art in article if '§' not in art]))
    article = natsort.natsorted(article)
    articles.append(article)

In [61]:
conhecimento_externo = []
for article_group in articles:
    row_knowledge = [get_article(article) for article in article_group]
    print(row_knowledge)
    conhecimento_externo.append('\n'.join(row_knowledge).replace('\n\n', '\n').strip())

df['rag_artigos_candidatos'] = conhecimento_externo

['Art. 1.815. A exclusão do herdeiro ou legatário, em qualquer desses casos de indignidade, será declarada por sentença.Parágrafo único. O direito de demandar a exclusão do herdeiro ou legatário extingue-se em quatro anos, contados da abertura da sucessão.§ 1 o O direito de demandar a exclusão do herdeiro ou legatário extingue-se em quatro anos, contados da abertura da sucessão.     (Redação dada pela Lei nº 13.532, de 2017)§ 2 o Na hipótese do inciso I do art. 1.814, o Ministério Público tem legitimidade para demandar a exclusão do herdeiro ou legatário.     (Incluído pela Lei nº 13.532, de 2017)  \nArt. 1.815-A. Em qualquer dos casos de indignidade previstos no art. 1.814, o trânsito em julgado da sentença penal condenatória acarretará a imediata exclusão do herdeiro ou legatário indigno, independentemente da sentença prevista no caput do art. 1.815 deste Código.    (Incluído pela Lei nº 14.661, de 2023)', 'Art. 1.816. São pessoais os efeitos da exclusão; os descendentes do herdeiro 

In [58]:
conhecimento_externo = []
for article_group in articles:
    row_knowledge = list(set(['\n'.join(get_article_group(article, documents)) for article in article_group]))
    conhecimento_externo.append('\n'.join(row_knowledge).replace('\n\n', '\n').strip())

df['rag_artigos_candidatos_grupos'] = conhecimento_externo

In [62]:
print(df['rag_artigos_candidatos'].iloc[3])

Art. 1.848. Salvo se houver justa causa, declarada no testamento, não pode o testador estabelecer cláusula de inalienabilidade, impenhorabilidade, e de incomunicabilidade, sobre os bens da legítima.§ 1 o Não é permitido ao testador estabelecer a conversão dos bens da legítima em outros de espécie diversa.§ 2 o Mediante autorização judicial e havendo justa causa, podem ser alienados os bens gravados, convertendo-se o produto em outros bens, que ficarão sub-rogados nos ônus dos primeiros.
Art. 1.857. Toda pessoa capaz pode dispor, por testamento, da totalidade dos seus bens, ou de parte deles, para depois de sua morte.§ 1 o A legítima dos herdeiros necessários não poderá ser incluída no testamento.§ 2 o São válidas as disposições testamentárias de caráter não patrimonial, ainda que o testador somente a elas se tenha limitado.
Art. 1.860. Além dos incapazes, não podem testar os que, no ato de fazê-lo, não tiverem pleno discernimento.Parágrafo único. Podem testar os maiores de dezesseis an

In [60]:
print(df['rag_artigos_candidatos_grupos'].iloc[3])

Art. 1.860. Além dos incapazes, não podem testar os que, no ato de fazê-lo, não tiverem pleno discernimento.
Parágrafo único. Podem testar os maiores de dezesseis anos.
  
Art. 1.861. A incapacidade superveniente do testador não invalida o testamento, nem o testamento do incapaz se valida com a superveniência da capacidade.

Art. 1.857. Toda pessoa capaz pode dispor, por testamento, da totalidade dos seus bens, ou de parte deles, para depois de sua morte.
§ 1 o A legítima dos herdeiros necessários não poderá ser incluída no testamento.
§ 2 o São válidas as disposições testamentárias de caráter não patrimonial, ainda que o testador somente a elas se tenha limitado.
  
Art. 1.858. O testamento é ato personalíssimo, podendo ser mudado a qualquer tempo.
  
Art. 1.859. Extingue-se em cinco anos o direito de impugnar a validade do testamento, contado o prazo da data do seu registro.

Art. 1.845. São herdeiros necessários os descendentes, os ascendentes e o cônjuge.
  
Art. 1.846. Pertence ao

In [63]:
df.to_csv("dataset_concurso_rag.csv", index=False)
!cp dataset_concurso_rag.csv /content/drive/MyDrive/

## Analisando

In [84]:
df.to_csv("dataset_concurso_rag.csv", index=False)
!cp dataset_concurso_rag.csv /content/drive/MyDrive/

In [83]:
df[['rag_art_inventado_full', 'rag_art_inventado_full_sem_art', 'rag_assunto_full', 'rag_topicos_full', 'rag_artigos_candidatos','rag_artigos_candidatos_grupos']]

,rag_art_inventado_full,rag_art_inventado_full_sem_art,rag_assunto_full,rag_topicos_full,rag_artigos_candidatos,rag_artigos_candidatos_grupos
0,"Art. 1.238. Aquele que, por quinze anos, sem i...","Art. 1.238. Aquele que, por quinze anos, sem i...","Art. 1.824. O herdeiro pode, em ação de petiçã...",Art. 1.210. O possuidor tem direito a ser mant...,Art. 1.815. A exclusão do herdeiro ou legatári...,Art. 1.814. São excluídos da sucessão os herde...
1,Art. 1.976. O testador pode nomear um ou mais ...,Art. 1.976. O testador pode nomear um ou mais ...,Art. 26. Decorrido um ano da arrecadação dos b...,"Art. 1.784. Aberta a sucessão, a herança trans...",Art. 1.862. São testamentos ordinários:I - o p...,Art. 1.862. São testamentos ordinários:\nI - o...
2,Art. 1.876. O testamento particular pode ser e...,Art. 1.876. O testamento particular pode ser e...,"Art. 1.857. Toda pessoa capaz pode dispor, por...",Art. 212. Salvo o negócio a que se impõe forma...,"Art. 1.857. Toda pessoa capaz pode dispor, por...",Art. 1.862. São testamentos ordinários:\nI - o...
3,Art. 1.976. O testador pode nomear um ou mais ...,Art. 1.976. O testador pode nomear um ou mais ...,Art. 1.976. O testador pode nomear um ou mais ...,Art. 92. Principal é o bem que existe sobre si...,"Art. 1.848. Salvo se houver justa causa, decla...","Art. 1.860. Além dos incapazes, não podem test..."
4,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,"Art. 1.784. Aberta a sucessão, a herança trans..."
5,"Art. 1.022. A sociedade adquire direitos, assu...","Art. 1.022. A sociedade adquire direitos, assu...","Art. 1.022. A sociedade adquire direitos, assu...",Art. 986. Enquanto não inscritos os atos const...,"Art. 1.793. O direito à sucessão aberta, bem c...",Art. 1.791. A herança defere-se como um todo u...
6,"Art. 1.596. Os filhos, havidos ou não da relaç...","Art. 1.596. Os filhos, havidos ou não da relaç...","Art. 1.658. No regime de comunhão parcial, com...","Art. 1.639. É lícito aos nubentes, antes de ce...",Art. 1.576. A separação judicial põe termo aos...,"Art. 1.596. Os filhos, havidos ou não da relaç..."
7,Art. 1.997. A herança responde pelo pagamento ...,Art. 1.997. A herança responde pelo pagamento ...,"Art. 789. Nos seguros de pessoas, o capital se...","Art. 818. Pelo contrato de fiança, uma pessoa ...",Art. 1.792. O herdeiro não responde por encarg...,Art. 1.791. A herança defere-se como um todo u...
8,Art. 26. Decorrido um ano da arrecadação dos b...,Art. 26. Decorrido um ano da arrecadação dos b...,"Art. 1.824. O herdeiro pode, em ação de petiçã...","Art. 1.824. O herdeiro pode, em ação de petiçã...",Art. 1.813. Quando o herdeiro prejudicar os se...,Art. 1.814. São excluídos da sucessão os herde...
9,"Art. 1.857. Toda pessoa capaz pode dispor, por...","Art. 1.857. Toda pessoa capaz pode dispor, por...",Art. 26. Decorrido um ano da arrecadação dos b...,Art. 1.798. Legitimam-se a suceder as pessoas ...,Art. 1.830. Somente é reconhecido direito suce...,Art. 1.845. São herdeiros necessários os desce...


# Obtendo trechos do dataset

In [118]:
class GET_CONTEXT_JSON(BaseModel):
    analise: str = Field(description="Análise da questão, resposta e artigos de contexto.")
    parte_dos_artigos: List[str] = Field(description="Porçao ou trecho do artigo que ajuda a responder a pergunta.")
    artigos_id: List[str] = Field(description="Nome do artigos cujos trechos respondem a pergunta.")
    justificativa_de_cada_trecho: List[str] = Field(description="Razões pelas quais os trechos ajudam a responder a pergunta.")
    informacao_faltante: str = Field(description="Informação faltante que impede a resolução correta da questão, se houver.")
    resposta_obtida: str = Field(description="Resposta obtida pela análise dos trechos selecionados.")
    alternativa_resposta_obtida: str = Field(description="Alternativa da resposta obtida: A, B, C, D ou E.")
    resposta_correta: str = Field(description="Sim, se a resposta obtida for igual a resposta oficial.")


output_parser = JsonOutputParser(pydantic_object=GET_CONTEXT_JSON)

In [126]:
prompt = """
Você é professor de Direito e ávido conhecedor do Código Civil Brasileiro.
Use seus conhecimentos em leis e códigos analisar os itens da questão.

Em seguida selecione trechos dos artigos que melhor resolvem as sentenças da questão.

Irei fornecer uma lista de possíveis artigos do Código Civil e uma outra lista de artigos
citados por estudantes.

Os trechos devem ser usados para CONFIRMAR ou NEGAR as sentenças da questão.

Depois de gerar os trechos, análise se eles ajudam a chegar na resposta correta.
Caso não possibilitem chegar na mesma resposta, indique que informação está faltando.

questão: {questão}
resposta correta: {resposta}
artigos estudantes: {artigos_estudantes}
artigos possíveis: {artigos_da_busca}

Gere somente as informações dos campos do JSON, sem introduções ou formalismos.
Instruções de saída: {format_instructions}
""".strip()

prompt_template = PromptTemplate(
    input_variables=["questão", "resposta", "artigos_estudantes", "artigos_da_busca"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [127]:
df.columns

Index(['pergunta', 'resposta', 'comentarios', 'numero_na_captura', 'id',
       'area', 'ano', 'banca', 'prova', 'duvidou_do_gabarito', 'enunciado',
       'assertivas', 'alternativas', 'classificacao',
       'artigos_nos_comentarios', 'artigos_completos', 'artigo_inventado',
       'topicos_para_busca', 'artigos_candidatos', 'frase_assunto_da_questao',
       'rag_art_inventado_full', 'rag_art_inventado_full_sem_art',
       'rag_assunto_full', 'rag_topicos_full', 'rag_artigos_candidatos',
       'rag_artigos_candidatos_grupos'],
      dtype='object')

In [136]:
def call_context_generator(sample, rag_column):
    output = chain.invoke(
        {"questão": sample['pergunta'],
         "resposta": sample['resposta'],
         "artigos_estudantes": sample['artigos_completos'],
         "artigos_da_busca": sample[rag_column],
         })
    return output

# test
out = call_context_generator(df.iloc[9], rag_column='rag_art_inventado_full')
out

{'analise': 'A questão apresenta uma situação em que Diana, devedora do Banco XYZ, renuncia à herança deixada por sua tia Judith para evitar que o banco se aproprie de toda a herança. O banco, no entanto, pode aceitar a herança em nome de Diana e receber um quarto do monte deixado por Judith.',
 'parte_dos_artigos': ['§ 1 o A habilitação dos credores se fará no prazo de trinta dias seguintes ao conhecimento do fato.',
  '§ 2 o Pagas as dívidas do renunciante, prevalece a renúncia quanto ao remanescente, que será devolvido aos demais herdeiros.'],
 'artigos_id': ['Art. 1.813'],
 'justificativa_de_cada_trecho': ['O trecho do art. 1.813, § 1 o, estabelece o prazo de 30 dias para os credores se habilitarem após o conhecimento do fato, o que permite ao Banco XYZ aceitar a herança em nome de Diana.',
  'O trecho do art. 1.813, § 2 o, estabelece que, após o pagamento das dívidas do renunciante, a renúncia prevalece quanto ao remanescente, que será devolvido aos demais herdeiros.'],
 'informac

In [117]:
print(df.iloc[2]['pergunta'])

Matheus, filho de um milionário, aos dezesseis anos, decide escrever, de próprio punho, seu testamento. No testamento, escrito em língua francesa, Matheus decide dispor da totalidade de seus bens. Passados cinco meses, mediante escrito particular seu, datado e assinado, Matheus decide fazer disposições especiais sobre o seu enterro. Além disso, decide alterar seu testamento, dispondo apenas de seus bens imóveis. Passados dois meses, decide novamente alterar seu testamento, dispondo novamente da totalidade de seus bens. Tanto a elaboração do testamento quanto suas retificações foram lidas e assinadas por Matheus na presença de três testemunhas, que o subscreveram. Diante da situação hipotética, assinale a alternativa correta.
A) O codicilo só é válido se não houver testamento; havendo testamento, as disposições de última vontade deverão nele ser incluídas.
B) O testamento não é válido, sendo necessária a maioridade para sua validade ou a participação do Ministério Público, enquanto meno

## Loop rag_art_inventado_full

In [140]:
list_of_outputs = loop_fn(df, call_context_generator, kwarg='rag_art_inventado_full')

  3%|▎         | 1/37 [00:03<02:12,  3.69s/it]

{'analise': 'Diego, herdeiro aparente e de boa-fé, tem direito a todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias necessárias e úteis.', 'parte_dos_artigos': ['Art. 1.214. O possuidor de boa-fé tem direito, enquanto ela durar, aos frutos percebidos.', 'Art. 1.219. O possuidor de boa-fé tem direito à indenização das benfeitorias necessárias e úteis, bem como, quanto às voluptuárias, se não lhe forem pagas, a levantá-las, quando o puder sem detrimento da coisa, e poderá exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.'], 'artigos_id': ['Art. 1.214', 'Art. 1.219'], 'justificativa_de_cada_trecho': ['O art. 1.214 estabelece que o possuidor de boa-fé tem direito aos frutos percebidos enquanto durar a boa-fé.', 'O art. 1.219 estabelece que o possuidor de boa-fé tem direito à indenização das benfeitorias necessárias

  5%|▌         | 2/37 [00:06<01:57,  3.35s/it]

{'analise': 'A questão apresenta quatro afirmativas sobre a sucessão testamentária. Afirmativa I está correta, pois o Código Civil Brasileiro estabelece que ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento. Afirmativa IV também está correta, pois o testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a compreendam.', 'parte_dos_artigos': ['Art. 1.867. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.', 'Art. 1.880. O testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a compreendam.'], 'artigos_id': ['Art. 1.867', '

  8%|▊         | 3/37 [00:58<14:24, 25.43s/it]

...

{'analise': 'A legítima dos herdeiros necessários não pode ser incluída no testamento.', 'parte_dos_artigos': ['§ 1 o A legítima dos herdeiros necessários não poderá ser incluída no testamento.'], 'artigos_id': ['Art. 1.857'], 'justificativa_de_cada_trecho': ['Este artigo estabelece que a legítima dos herdeiros necessários não pode ser incluída no testamento.'], 'informacao_faltante': '', 'resposta_obtida': 'Certo', 'alternativa_resposta_obtida': 'C', 'resposta_correta': 'Sim'}


 97%|█████████▋| 36/37 [22:01<00:33, 33.22s/it]

{'analise': 'A petição de herança de Renato está prescrita, pois o prazo prescricional é de 10 anos, contados da abertura da sucessão, e Renato só ajuizou a ação em 2022, mais de 12 anos após a abertura da sucessão.', 'parte_dos_artigos': ['Art. 205. A prescrição ocorre em dez anos, quando a lei não lhe haja fixado prazo menor.', 'Art. 1.822. A declaração de vacância da herança não prejudicará os herdeiros que legalmente se habilitarem; mas, decorridos cinco anos da abertura da sucessão, os bens arrecadados passarão ao domínio do Município ou do Distrito Federal, se localizados nas respectivas circunscrições, incorporando-se ao domínio da União quando situados em território federal.'], 'artigos_id': ['Art. 205', 'Art. 1.822'], 'justificativa_de_cada_trecho': ['O art. 205 estabelece o prazo prescricional de 10 anos para a petição de herança.', 'O art. 1.822 estabelece que, decorridos 5 anos da abertura da sucessão, os bens arrecadados passarão ao domínio do Município ou do Distrito Fede

100%|██████████| 37/37 [22:39<00:00, 36.74s/it]

{'analise': 'A questão apresenta uma situação hipotética em que João morre e sua esposa Ana está grávida. A pergunta é se o nascituro tem legitimidade para suceder.', 'parte_dos_artigos': ['Legitimam-se a suceder as pessoas nascidas ou já concebidas no momento da abertura da sucessão.'], 'artigos_id': ['Art. 1.798'], 'justificativa_de_cada_trecho': ['Este artigo estabelece que as pessoas concebidas no momento da abertura da sucessão têm legitimidade para suceder.'], 'informacao_faltante': '', 'resposta_obtida': 'Sim, o nascituro tem legitimidade para suceder.', 'alternativa_resposta_obtida': 'C', 'resposta_correta': 'Sim'}


In [144]:
out = call_context_generator(df.iloc[15], rag_column='rag_art_inventado_full')
out

{'analise': 'A questão apresentada refere-se ao direito de alimentos no âmbito do direito de família.',
 'parte_dos_artigos': ['Art. 1.696. O direito à prestação de alimentos é recíproco entre pais e filhos, e extensivo a todos os ascendentes, recaindo a obrigação nos mais próximos em grau, uns em falta de outros.',
  'Art. 1.698. Se o parente, que deve alimentos em primeiro lugar, não estiver em condições de suportar totalmente o encargo, serão chamados a concorrer os de grau imediato; sendo várias as pessoas obrigadas a prestar alimentos, todas devem concorrer na proporção dos respectivos recursos, e, intentada ação contra uma delas, poderão as demais ser chamadas a integrar a lide.'],
 'artigos_id': ['Art. 1.696', 'Art. 1.698'],
 'justificativa_de_cada_trecho': ['Este artigo estabelece a reciprocidade do direito à prestação de alimentos entre pais e filhos, e sua extensão a todos os ascendentes.',
  'Este artigo determina que, se o parente que deve alimentos em primeiro lugar não es

In [145]:
list_of_outputs[15] = out

In [146]:
out = call_context_generator(df.iloc[22], rag_column='rag_art_inventado_full')
out

{'analise': 'A questão apresentada refere-se à exclusão de um herdeiro por indignidade, especificamente no caso de Pedro, que praticou crime contra a honra do companheiro de Ana pouco antes do falecimento da autora da herança.',
 'parte_dos_artigos': ['Art. 1.814. São excluídos da sucessão os herdeiros ou legatários:',
  'Art. 1.815. A exclusão do herdeiro ou legatário, em qualquer desses casos de indignidade, será declarada por sentença.',
  'Art. 1.816. São pessoais os efeitos da exclusão; os descendentes do herdeiro excluído sucedem, como se ele morto fosse antes da abertura da sucessão.'],
 'artigos_id': ['Art. 1.814', 'Art. 1.815', 'Art. 1.816'],
 'justificativa_de_cada_trecho': ['O art. 1.814 estabelece as hipóteses de indignidade que excluem os herdeiros ou legatários da sucessão.',
  'O art. 1.815 determina que a exclusão do herdeiro ou legatário será declarada por sentença.',
  'O art. 1.816 estabelece que os efeitos da exclusão são pessoais e que os descendentes do herdeiro e

In [147]:
list_of_outputs[22] = out

In [148]:
out = call_context_generator(df.iloc[30], rag_column='rag_art_inventado_full')
out

{'analise': 'A questão apresentada trata de uma menor de idade que cometeu um ato infracional análogo ao homicídio doloso contra seus pais, visando herdar a fortuna deles. A discussão gira em torno da possibilidade de exclusão da menor da sucessão, considerando a idade e a natureza do ato cometido.',
 'parte_dos_artigos': ['Art. 1.814. São excluídos da sucessão os herdeiros ou legatários:',
  'Art. 1.816. São pessoais os efeitos da exclusão; os descendentes do herdeiro excluído sucedem, como se ele morto fosse antes da abertura da sucessão.'],
 'artigos_id': ['Art. 1.814', 'Art. 1.816'],
 'justificativa_de_cada_trecho': ['O art. 1.814 estabelece as hipóteses de exclusão da sucessão, entre as quais se encontra o homicídio doloso contra os autores da herança.',
  'O art. 1.816 determina que os efeitos da exclusão são pessoais e que os descendentes do herdeiro excluído sucedem como se ele fosse morto antes da abertura da sucessão.'],
 'informacao_faltante': 'Não há informação faltante que

In [149]:
list_of_outputs[30] = out

In [150]:
df_rag_art_inventado_full = pd.DataFrame(list_of_outputs)
df_rag_art_inventado_full.to_csv("df_rag_art_inventado_full.csv", index=False)
!cp df_rag_art_inventado_full.csv /content/drive/MyDrive/

In [151]:
df_rag_art_inventado_full

,analise,parte_dos_artigos,artigos_id,justificativa_de_cada_trecho,informacao_faltante,resposta_obtida,alternativa_resposta_obtida,resposta_correta
0,"Diego, herdeiro aparente e de boa-fé, tem dire...",[Art. 1.214. O possuidor de boa-fé tem direito...,"[Art. 1.214, Art. 1.219]",[O art. 1.214 estabelece que o possuidor de bo...,,"C) todos os frutos percebidos até a citação, b...",C,Sim
1,A questão apresenta quatro afirmativas sobre a...,[Art. 1.867. Ao cego só se permite o testament...,"[Art. 1.867, Art. 1.880]",[Este artigo estabelece as formalidades para o...,,B) I e IV.,B,Sim
2,A questão apresenta um caso de testamento feit...,[Art. 1.880. O testamento particular pode ser ...,"[Art. 1.880, Art. 1.876]",[O artigo 1.880 permite que o testamento seja ...,,"O testamento é válido, desde que as testemunha...",E,Sim
3,A questão apresenta uma situação hipotética em...,"[Art. 1.860. Além dos incapazes, não podem tes...","[Art. 1.860, Art. 1.861, Art. 1.799]",[O art. 1.860 estabelece que os relativamente ...,,A incapacidade relativa de Mariana no momento ...,C,Sim
4,A situação hipotética apresentada configura um...,[§ 1 o A união estável não se constituirá se o...,[Art. 1.723],"[O trecho do art. 1.723, § 1º, justifica que a...",,Caso Rogério estivesse separado de fato de Clá...,C,Sim
5,A questão discute a responsabilidade dos herde...,[nenhum destes será obrigado a pagar senão a q...,[Art. 276],[Este trecho explica que os herdeiros de João ...,,E,Errado,Sim
6,A questão apresenta um caso de divórcio culpos...,[Art. 1.571. A sociedade conjugal termina: ......,"[Art. 1.571, Art. 1.573, Art. 1.601]",[O art. 1.571 estabelece que a sociedade conju...,,E,E,Sim
7,A questão discute a responsabilidade dos herde...,[Art. 270. Se um dos credores solidários falec...,[Art. 270],"[Este artigo estabelece que, se um dos credore...",,Errado,E,Sim
8,"A questão apresenta uma situação em que Diana,...",[§ 1 o A habilitação dos credores se fará no p...,[Art. 1.813],"[O trecho do art. 1.813, § 1 o, estabelece o p...",,D) poderá aceitar a herança e receber um quart...,D,Sim
9,"Análise da questão de Direito das Sucessões, q...","[Art. 1.857. § 1o, Art. 1.858., Art. 1.786., A...","[Art. 1.857, Art. 1.858, Art. 1.786, Art. 1.84...",[O menor de dezoito anos pode fazer testamento...,,"B) O testamento é ato personalíssimo, podendo ...",B,Sim


## Loop rag_art_inventado_full_sem_art

In [152]:
list_of_outputs = loop_fn(df, call_context_generator, kwarg='rag_art_inventado_full_sem_art')

  3%|▎         | 1/37 [00:02<01:43,  2.89s/it]

{'analise': 'Diego, herdeiro aparente e de boa-fé, tem direito a todos os frutos já percebidos, bem como aqueles que forem percebidos antes da prolação eventual sentença de procedência dos pedidos de João Carlos e, também tem direito à indenização das benfeitorias necessárias e úteis.', 'parte_dos_artigos': ['Art. 1.214. O possuidor de boa-fé tem direito, enquanto ela durar, aos frutos percebidos.', 'Art. 1.219. O possuidor de boa-fé tem direito à indenização das benfeitorias necessárias e úteis, bem como, quanto às voluptuárias, se não lhe forem pagas, a levantá-las, quando o puder sem detrimento da coisa, e poderá exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.'], 'artigos_id': ['Art. 1.214', 'Art. 1.219'], 'justificativa_de_cada_trecho': ['O art. 1.214 estabelece que o possuidor de boa-fé tem direito aos frutos percebidos enquanto durar a boa-fé.', 'O art. 1.219 estabelece que o possuidor de boa-fé tem direito à indenização das benfeitorias necessárias

  5%|▌         | 2/37 [00:05<01:32,  2.64s/it]

{'analise': 'A questão apresenta quatro afirmativas sobre a sucessão testamentária. Afirmativa I está correta, pois o Código Civil Brasileiro estabelece que ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento. Afirmativa IV também está correta, pois o testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a compreendam.', 'parte_dos_artigos': ['Art. 1.867. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.', 'Art. 1.880. O testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a compreendam.'], 'artigos_id': ['Art. 1.867', '

  8%|▊         | 3/37 [00:56<13:56, 24.60s/it]

...

{'analise': 'A legítima dos herdeiros necessários não pode ser incluída no testamento.', 'parte_dos_artigos': ['§ 1 o A legítima dos herdeiros necessários não poderá ser incluída no testamento.'], 'artigos_id': ['Art. 1.857'], 'justificativa_de_cada_trecho': ['Este artigo estabelece que a legítima dos herdeiros necessários não pode ser incluída no testamento.'], 'informacao_faltante': '', 'resposta_obtida': 'Certo', 'alternativa_resposta_obtida': 'C', 'resposta_correta': 'Sim'}


 97%|█████████▋| 36/37 [22:00<00:33, 33.25s/it]

{'analise': 'A petição de herança de Renato está prescrita, pois o prazo prescricional é de 10 anos, contados da abertura da sucessão, e Renato só ajuizou a ação em 2022, mais de 12 anos após a abertura da sucessão.', 'parte_dos_artigos': ['Art. 205. A prescrição ocorre em dez anos, quando a lei não lhe haja fixado prazo menor.', 'Art. 1.822. A declaração de vacância da herança não prejudicará os herdeiros que legalmente se habilitarem; mas, decorridos cinco anos da abertura da sucessão, os bens arrecadados passarão ao domínio do Município ou do Distrito Federal, se localizados nas respectivas circunscrições, incorporando-se ao domínio da União quando situados em território federal.'], 'artigos_id': ['Art. 205', 'Art. 1.822'], 'justificativa_de_cada_trecho': ['O art. 205 estabelece o prazo prescricional de 10 anos para a petição de herança.', 'O art. 1.822 estabelece que, decorridos 5 anos da abertura da sucessão, os bens arrecadados passarão ao domínio do Município ou do Distrito Fede

100%|██████████| 37/37 [22:38<00:00, 36.71s/it]

{'analise': 'A questão apresenta uma situação hipotética em que João morre e sua esposa Ana está grávida. A pergunta é se o nascituro tem legitimidade para suceder.', 'parte_dos_artigos': ['Legitimam-se a suceder as pessoas nascidas ou já concebidas no momento da abertura da sucessão.'], 'artigos_id': ['Art. 1.798'], 'justificativa_de_cada_trecho': ['Este artigo estabelece que as pessoas concebidas no momento da abertura da sucessão têm legitimidade para suceder.'], 'informacao_faltante': '', 'resposta_obtida': 'Sim, o nascituro tem legitimidade para suceder.', 'alternativa_resposta_obtida': 'C', 'resposta_correta': 'Sim'}


In [154]:
list_of_outputs[15], list_of_outputs[22], list_of_outputs[30]

('Error', 'Error', 'Error')

In [155]:
out = call_context_generator(df.iloc[15], rag_column='rag_art_inventado_full_sem_art')
out

{'analise': 'A questão apresentada refere-se ao direito de alimentos no âmbito do direito de família.',
 'parte_dos_artigos': ['Art. 1.696. O direito à prestação de alimentos é recíproco entre pais e filhos, e extensivo a todos os ascendentes, recaindo a obrigação nos mais próximos em grau, uns em falta de outros.',
  'Art. 1.698. Se o parente, que deve alimentos em primeiro lugar, não estiver em condições de suportar totalmente o encargo, serão chamados a concorrer os de grau imediato; sendo várias as pessoas obrigadas a prestar alimentos, todas devem concorrer na proporção dos respectivos recursos, e, intentada ação contra uma delas, poderão as demais ser chamadas a integrar a lide.'],
 'artigos_id': ['Art. 1.696', 'Art. 1.698'],
 'justificativa_de_cada_trecho': ['Este artigo estabelece a reciprocidade do direito à prestação de alimentos entre pais e filhos, e sua extensão a todos os ascendentes.',
  'Este artigo determina que, se o parente que deve alimentos em primeiro lugar não es

In [156]:
list_of_outputs[15] = out

In [157]:
out = call_context_generator(df.iloc[22], rag_column='rag_art_inventado_full_sem_art')
out

{'analise': 'A questão apresentada refere-se à exclusão de um herdeiro por indignidade, especificamente no caso de Pedro, que praticou crime contra a honra do companheiro de Ana pouco antes do falecimento da autora da herança.',
 'parte_dos_artigos': ['Art. 1.814. São excluídos da sucessão os herdeiros ou legatários:',
  'Art. 1.815. A exclusão do herdeiro ou legatário, em qualquer desses casos de indignidade, será declarada por sentença.',
  'Art. 1.816. São pessoais os efeitos da exclusão; os descendentes do herdeiro excluído sucedem, como se ele morto fosse antes da abertura da sucessão.'],
 'artigos_id': ['Art. 1.814', 'Art. 1.815', 'Art. 1.816'],
 'justificativa_de_cada_trecho': ['O art. 1.814 estabelece as hipóteses de exclusão de herdeiros ou legatários, incluindo a prática de crime contra a honra do autor da herança ou de seu cônjuge ou companheiro.',
  'O art. 1.815 determina que a exclusão do herdeiro ou legatário será declarada por sentença.',
  'O art. 1.816 estabelece que 

In [158]:
list_of_outputs[22] = out

In [159]:
out = call_context_generator(df.iloc[30], rag_column='rag_art_inventado_full_sem_art')
out

{'analise': 'A questão apresentada trata de uma menor de idade que cometeu um ato infracional análogo ao homicídio doloso contra seus pais, visando herdar a fortuna deles. A discussão gira em torno da possibilidade de exclusão da menor da sucessão, considerando a idade e a natureza do ato cometido.',
 'parte_dos_artigos': ['Art. 1.814. São excluídos da sucessão os herdeiros ou legatários:',
  'Art. 1.816. São pessoais os efeitos da exclusão; os descendentes do herdeiro excluído sucedem, como se ele morto fosse antes da abertura da sucessão.'],
 'artigos_id': ['Art. 1.814', 'Art. 1.816'],
 'justificativa_de_cada_trecho': ['O art. 1.814 estabelece as hipóteses de exclusão da sucessão, entre as quais se encontra o homicídio doloso contra os autores da herança.',
  'O art. 1.816 determina que os efeitos da exclusão são pessoais e que os descendentes do herdeiro excluído sucedem como se ele fosse morto antes da abertura da sucessão.'],
 'informacao_faltante': 'Não há informação faltante que

In [160]:
list_of_outputs[30] = out

In [161]:
df_rag_art_inventado_full_sem_art = pd.DataFrame(list_of_outputs)
df_rag_art_inventado_full_sem_art.to_csv("df_rag_art_inventado_full_sem_art.csv", index=False)
!cp df_rag_art_inventado_full_sem_art.csv /content/drive/MyDrive/

In [162]:
df_rag_art_inventado_full_sem_art

,analise,parte_dos_artigos,artigos_id,justificativa_de_cada_trecho,informacao_faltante,resposta_obtida,alternativa_resposta_obtida,resposta_correta
0,"Diego, herdeiro aparente e de boa-fé, tem dire...",[Art. 1.214. O possuidor de boa-fé tem direito...,"[Art. 1.214, Art. 1.219]",[O art. 1.214 estabelece que o possuidor de bo...,,"C) todos os frutos percebidos até a citação, b...",C,Sim
1,A questão apresenta quatro afirmativas sobre a...,[Art. 1.867. Ao cego só se permite o testament...,"[Art. 1.867, Art. 1.880]",[Este artigo estabelece as formalidades para o...,,B) I e IV.,B,Sim
2,A questão apresenta um caso de testamento feit...,[Art. 1.880. O testamento particular pode ser ...,"[Art. 1.880, Art. 1.876]",[O artigo 1.880 permite que o testamento seja ...,,"O testamento é válido, desde que as testemunha...",E,Sim
3,A questão apresenta uma situação hipotética em...,"[Art. 1.860. Além dos incapazes, não podem tes...","[Art. 1.860, Art. 1.861, Art. 1.799]",[O art. 1.860 estabelece que os relativamente ...,,A incapacidade relativa de Mariana no momento ...,C,Sim
4,A situação hipotética apresentada configura um...,[§ 1 o A união estável não se constituirá se o...,[Art. 1.723],[Este trecho do artigo 1.723 estabelece que a ...,,"Sim, é possível a configuração da união estáve...",C,Sim
5,A questão discute a responsabilidade dos herde...,[nenhum destes será obrigado a pagar senão a q...,[Art. 276],[Este trecho explica que os herdeiros de João ...,,E,Errado,Sim
6,A questão apresenta um caso de divórcio culpos...,[Art. 1.571. A sociedade conjugal termina: ......,"[Art. 1.571, Art. 1.573, Art. 1.580]",[O art. 1.571 estabelece que o divórcio é uma ...,,E,E,Sim
7,A questão discute a responsabilidade dos herde...,[Art. 270. Se um dos credores solidários falec...,[Art. 270],"[Este artigo estabelece que, em caso de faleci...",,Errado,E,Sim
8,"A questão apresenta uma situação em que Diana,...",[§ 1 o A habilitação dos credores se fará no p...,[Art. 1.813],[O art. 1.813 estabelece que os credores podem...,,O Banco XYZ pode aceitar a herança e receber u...,D,Sim
9,"Análise da questão de Direito das Sucessões, q...",[Art. 1.858. O testamento é ato personalíssimo...,"[Art. 1.858, Art. 1.786, Art. 1.860]",[O art. 1.858 confirma que o testamento é ato ...,,"B) O testamento é ato personalíssimo, podendo ...",B,Sim
